# Ordering by metrics and retraining phase

## Dataset: Intel

## Experiment configuration 1


## Configuration 1
	1. Incremental guided retraining starting from scratch using the new adversarial inputs and original training set

In [ ]:
pip install --user tensorflow==2.5

In [1]:
import argparse

import numpy as np
import tensorflow as tf
import keras.backend as K

import matplotlib.pyplot as plt

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.regularizers import l2

import glob
import os
import cv2
import keras


import time
import argparse

from tqdm import tqdm

from keras.models import load_model, Model


In [2]:
tf.__version__ == '2.5.0' # Version of tensorflow

True

In [3]:
cd C:/Users/fjdur/Documents/utils/

C:\Users\fjdur\Documents\utils


In [4]:
# utils for project
import utils_guided_retraining as utils

2.5.0
keras
2.5.0


In [5]:
#cd '../notebooks/'

In [6]:
dataset = "cifar"

In [7]:
x_train,y_train = utils.get_data(dataset,"Train")
x_val,y_val = utils.get_data(dataset,"Val")
x_test,y_test = utils.get_data(dataset,"Test")

In [8]:
x_train_and_adversary,y_train_and_adversary = utils.get_data(dataset,"Train_and_adversary")

In [9]:
x_adversary_training = x_train_and_adversary[len(x_train):]
print(len(x_adversary_training))
y_adversary_training = y_train_and_adversary[len(y_train):]

print(len(y_adversary_training))

6000
6000


In [10]:
# Obtaining adversarial examples for testing 
x_test_and_adversary,y_test_and_adversary = utils.get_adversarial_data(dataset,'Test_adversarial') 

In [11]:
x_adversary_test = x_test_and_adversary[len(x_test):]
print(len(x_adversary_test))
y_adversary_test = y_test_and_adversary[len(y_test):]

print(len(y_adversary_test))

6000
6000


## ----

In [12]:
# Original model 

model_dir = "C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06/"
if(dataset == 'gtsrb'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06"
elif(dataset == 'intel'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/intel_model_21_10"
elif(dataset == 'mnist'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/models2"
elif(dataset == 'cifar'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/model_02"


In [13]:
print(model_dir)
model_original = utils.My_model(dataset,True, model_dir)

C:/Users/fjdur/Documents/upc-july/models/model_02
Model loaded correctly


In [14]:
print(x_train_and_adversary.shape)
len(x_train_and_adversary)//20

(47999, 32, 32, 1)


2399

In [15]:
n_data_points = 2400

## Loading LSA and DSA values 

In [16]:
lsa_direction = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/lsa_values.npy"

In [17]:
lsa_values = np.load(lsa_direction)


In [18]:

# Obtaining top n images by LSA values
top_images_by_lsa = utils.get_x_of_indexes(list(np.flip(np.argsort(lsa_values))),x_train_and_adversary)
top_labels_by_lsa = utils.get_x_of_indexes(list(np.flip(np.argsort(lsa_values))),y_train_and_adversary)


In [19]:
m = n_data_points
n = 0
image_sets_lsa = []
label_sets_lsa = []

# last
#for i in range(0,len(top_images_by_lsa)//m):

for i in range((len(top_images_by_lsa)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_lsa)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_lsa)%m))
        top_images_by_lsa_n = np.array(top_images_by_lsa[:n+m+(len(top_images_by_lsa)%m)])
        top_labels_by_lsa_n = np.array(top_labels_by_lsa[:n+m+(len(top_images_by_lsa)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_lsa_n = np.array(top_images_by_lsa[:n+m])
        top_labels_by_lsa_n = np.array(top_labels_by_lsa[:n+m])
    image_sets_lsa.append(top_images_by_lsa_n)
    label_sets_lsa.append(top_labels_by_lsa_n)
    print(len(top_images_by_lsa_n))
    n += m

0 :
0  ->  2400
2400
1 :
0  ->  4800
4800
2 :
0  ->  7200
7200
3 :
0  ->  9600
9600
4 :
0  ->  12000
12000
5 :
0  ->  14400
14400
6 :
0  ->  16800
16800
7 :
0  ->  19200
19200
8 :
0  ->  21600
21600
9 :
0  ->  24000
24000
10 :
0  ->  26400
26400
11 :
0  ->  28800
28800
12 :
0  ->  31200
31200
13 :
0  ->  33600
33600
14 :
0  ->  36000
36000
15 :
0  ->  38400
38400
16 :
0  ->  40800
40800
17 :
0  ->  43200
43200
18 :
0  ->  45600
45600
19 :
Last
0  ->  50399
47999


In [20]:
len(image_sets_lsa)

20

## Training guided by LSA

In [21]:
len(label_sets_lsa)

20

In [22]:
print(model_dir)

models_lsa = []
for i in range(len(label_sets_lsa)):
    print(i,":")
    model = utils.My_model(dataset,False,None)
    model.compile_model()
    models_lsa.append(model)


C:/Users/fjdur/Documents/upc-july/models/model_02
0 :
Model compiled
1 :
Model compiled
2 :
Model compiled
3 :
Model compiled
4 :
Model compiled
5 :
Model compiled
6 :
Model compiled
7 :
Model compiled
8 :
Model compiled
9 :
Model compiled
10 :
Model compiled
11 :
Model compiled
12 :
Model compiled
13 :
Model compiled
14 :
Model compiled
15 :
Model compiled
16 :
Model compiled
17 :
Model compiled
18 :
Model compiled
19 :
Model compiled


In [23]:
n = 0
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n=n+1

Epoch 1/20
38/38 [==============================] - 37s 257ms/step - loss: 2.2857 - accuracy: 0.1683 - val_loss: 2.2982 - val_accuracy: 0.0977
Epoch 2/20
38/38 [==============================] - 9s 239ms/step - loss: 2.1551 - accuracy: 0.2404 - val_loss: 2.1637 - val_accuracy: 0.1712
Epoch 3/20
38/38 [==============================] - 9s 240ms/step - loss: 2.0375 - accuracy: 0.2711 - val_loss: 2.0609 - val_accuracy: 0.2388
Epoch 4/20
38/38 [==============================] - 9s 239ms/step - loss: 1.9220 - accuracy: 0.2990 - val_loss: 1.9749 - val_accuracy: 0.2741
Epoch 5/20
38/38 [==============================] - 9s 240ms/step - loss: 1.8290 - accuracy: 0.3415 - val_loss: 1.8998 - val_accuracy: 0.2991
Epoch 6/20
38/38 [==============================] - 9s 243ms/step - loss: 1.7670 - accuracy: 0.3582 - val_loss: 1.8836 - val_accuracy: 0.3042
Epoch 7/20
38/38 [==============================] - 9s 242ms/step - loss: 1.6690 - accuracy: 0.3882 - val_loss: 1.9418 - val_accuracy: 0.2925
Epoch

In [24]:
n=1
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

Epoch 1/20
75/75 [==============================] - 18s 199ms/step - loss: 1.9901 - accuracy: 0.4460 - val_loss: 2.7001 - val_accuracy: 0.0976
Epoch 2/20
75/75 [==============================] - 14s 182ms/step - loss: 1.6953 - accuracy: 0.5045 - val_loss: 2.3104 - val_accuracy: 0.1025
Epoch 3/20
75/75 [==============================] - 14s 184ms/step - loss: 1.6048 - accuracy: 0.5052 - val_loss: 3.0944 - val_accuracy: 0.1005
Epoch 4/20
75/75 [==============================] - 14s 191ms/step - loss: 1.4930 - accuracy: 0.5256 - val_loss: 2.1953 - val_accuracy: 0.1671
Epoch 5/20
75/75 [==============================] - 14s 190ms/step - loss: 1.3958 - accuracy: 0.5410 - val_loss: 2.1523 - val_accuracy: 0.1941
Epoch 6/20
75/75 [==============================] - 13s 172ms/step - loss: 1.3131 - accuracy: 0.5644 - val_loss: 1.9875 - val_accuracy: 0.2569
Epoch 7/20
75/75 [==============================] - 13s 175ms/step - loss: 1.2620 - accuracy: 0.5893 - val_loss: 1.9856 - val_accuracy: 0.2686

In [25]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

2
Epoch 1/20
113/113 [==============================] - 21s 156ms/step - loss: 1.6704 - accuracy: 0.6015 - val_loss: 2.6925 - val_accuracy: 0.0976
Epoch 2/20
113/113 [==============================] - 18s 159ms/step - loss: 1.3604 - accuracy: 0.6298 - val_loss: 3.0099 - val_accuracy: 0.0976
Epoch 3/20
113/113 [==============================] - 18s 158ms/step - loss: 1.2034 - accuracy: 0.6399 - val_loss: 2.6760 - val_accuracy: 0.1149
Epoch 4/20
113/113 [==============================] - 18s 159ms/step - loss: 1.1496 - accuracy: 0.6460 - val_loss: 2.5996 - val_accuracy: 0.1039
Epoch 5/20
113/113 [==============================] - 18s 161ms/step - loss: 1.0463 - accuracy: 0.6789 - val_loss: 1.8894 - val_accuracy: 0.2960
Epoch 6/20
113/113 [==============================] - 18s 163ms/step - loss: 1.0077 - accuracy: 0.6831 - val_loss: 2.4730 - val_accuracy: 0.1787
Epoch 7/20
113/113 [==============================] - 18s 162ms/step - loss: 0.9698 - accuracy: 0.6931 - val_loss: 2.1294 - val_

In [26]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

3
Epoch 1/20
150/150 [==============================] - 28s 170ms/step - loss: 1.8036 - accuracy: 0.4457 - val_loss: 2.7083 - val_accuracy: 0.1538
Epoch 2/20
150/150 [==============================] - 25s 166ms/step - loss: 1.3166 - accuracy: 0.5947 - val_loss: 2.1100 - val_accuracy: 0.1763
Epoch 3/20
150/150 [==============================] - 25s 166ms/step - loss: 1.1404 - accuracy: 0.6485 - val_loss: 2.0929 - val_accuracy: 0.2130
Epoch 4/20
150/150 [==============================] - 25s 166ms/step - loss: 1.0366 - accuracy: 0.6830 - val_loss: 2.5588 - val_accuracy: 0.1884
Epoch 5/20
150/150 [==============================] - 25s 166ms/step - loss: 0.9389 - accuracy: 0.7166 - val_loss: 2.1692 - val_accuracy: 0.2208
Epoch 6/20
150/150 [==============================] - 25s 166ms/step - loss: 0.9077 - accuracy: 0.7207 - val_loss: 2.3017 - val_accuracy: 0.2286
Epoch 7/20
150/150 [==============================] - 25s 165ms/step - loss: 0.8383 - accuracy: 0.7347 - val_loss: 2.5243 - val_

In [27]:
print(n)#
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

4
Epoch 1/20
188/188 [==============================] - 33s 158ms/step - loss: 1.6751 - accuracy: 0.3993 - val_loss: 2.5937 - val_accuracy: 0.1517
Epoch 2/20
188/188 [==============================] - 30s 159ms/step - loss: 1.2685 - accuracy: 0.6188 - val_loss: 2.1318 - val_accuracy: 0.1937
Epoch 3/20
188/188 [==============================] - 30s 159ms/step - loss: 1.0825 - accuracy: 0.6673 - val_loss: 2.0797 - val_accuracy: 0.2224
Epoch 4/20
188/188 [==============================] - 30s 159ms/step - loss: 0.9446 - accuracy: 0.7069 - val_loss: 1.9000 - val_accuracy: 0.2598
Epoch 5/20
188/188 [==============================] - 31s 164ms/step - loss: 0.8822 - accuracy: 0.7201 - val_loss: 2.0180 - val_accuracy: 0.2673
Epoch 6/20
188/188 [==============================] - 33s 174ms/step - loss: 0.8253 - accuracy: 0.7419 - val_loss: 2.0555 - val_accuracy: 0.2833
Epoch 7/20
188/188 [==============================] - 33s 173ms/step - loss: 0.7675 - accuracy: 0.7525 - val_loss: 1.6852 - val_

In [28]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

5
Epoch 1/20
225/225 [==============================] - 43s 173ms/step - loss: 1.6846 - accuracy: 0.3724 - val_loss: 2.7761 - val_accuracy: 0.1962
Epoch 2/20
225/225 [==============================] - 38s 170ms/step - loss: 1.2662 - accuracy: 0.5681 - val_loss: 2.1360 - val_accuracy: 0.2252
Epoch 3/20
225/225 [==============================] - 38s 171ms/step - loss: 1.0744 - accuracy: 0.6447 - val_loss: 2.2318 - val_accuracy: 0.2821
Epoch 4/20
225/225 [==============================] - 38s 170ms/step - loss: 0.9439 - accuracy: 0.6861 - val_loss: 1.8077 - val_accuracy: 0.3388
Epoch 5/20
225/225 [==============================] - 38s 171ms/step - loss: 0.8706 - accuracy: 0.7216 - val_loss: 2.1167 - val_accuracy: 0.2842
Epoch 6/20
225/225 [==============================] - 38s 170ms/step - loss: 0.8243 - accuracy: 0.7327 - val_loss: 2.2034 - val_accuracy: 0.2916
Epoch 7/20
225/225 [==============================] - 38s 170ms/step - loss: 0.7582 - accuracy: 0.7550 - val_loss: 1.9328 - val_

In [29]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

6
Epoch 1/20
263/263 [==============================] - 44s 156ms/step - loss: 1.7328 - accuracy: 0.3544 - val_loss: 2.3607 - val_accuracy: 0.2067
Epoch 2/20
263/263 [==============================] - 42s 160ms/step - loss: 1.2435 - accuracy: 0.5761 - val_loss: 2.1461 - val_accuracy: 0.2677
Epoch 3/20
263/263 [==============================] - 42s 158ms/step - loss: 1.0296 - accuracy: 0.6611 - val_loss: 2.0407 - val_accuracy: 0.2874
Epoch 4/20
263/263 [==============================] - 41s 156ms/step - loss: 0.8899 - accuracy: 0.7069 - val_loss: 1.9643 - val_accuracy: 0.3338
Epoch 5/20
263/263 [==============================] - 40s 154ms/step - loss: 0.8111 - accuracy: 0.7318 - val_loss: 1.8902 - val_accuracy: 0.3512
Epoch 6/20
263/263 [==============================] - 41s 155ms/step - loss: 0.7716 - accuracy: 0.7484 - val_loss: 1.6376 - val_accuracy: 0.3980
Epoch 7/20
263/263 [==============================] - 41s 155ms/step - loss: 0.7044 - accuracy: 0.7720 - val_loss: 1.6314 - val_

In [30]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

7
Epoch 1/20
300/300 [==============================] - 50s 155ms/step - loss: 1.7680 - accuracy: 0.3475 - val_loss: 2.3659 - val_accuracy: 0.2652
Epoch 2/20
300/300 [==============================] - 46s 152ms/step - loss: 1.3033 - accuracy: 0.5550 - val_loss: 3.1825 - val_accuracy: 0.2373
Epoch 3/20
300/300 [==============================] - 46s 153ms/step - loss: 1.1070 - accuracy: 0.6424 - val_loss: 1.8861 - val_accuracy: 0.3359
Epoch 4/20
300/300 [==============================] - 46s 155ms/step - loss: 0.9583 - accuracy: 0.6933 - val_loss: 1.9419 - val_accuracy: 0.3393
Epoch 5/20
300/300 [==============================] - 46s 153ms/step - loss: 0.8893 - accuracy: 0.7105 - val_loss: 1.7747 - val_accuracy: 0.3667
Epoch 6/20
300/300 [==============================] - 46s 155ms/step - loss: 0.8328 - accuracy: 0.7356 - val_loss: 1.9783 - val_accuracy: 0.3605
Epoch 7/20
300/300 [==============================] - 46s 153ms/step - loss: 0.7700 - accuracy: 0.7517 - val_loss: 1.7009 - val_

In [31]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

8
Epoch 1/20
338/338 [==============================] - 60s 166ms/step - loss: 1.8196 - accuracy: 0.3288 - val_loss: 2.1006 - val_accuracy: 0.2577
Epoch 2/20
338/338 [==============================] - 56s 166ms/step - loss: 1.3439 - accuracy: 0.5480 - val_loss: 2.1001 - val_accuracy: 0.3089
Epoch 3/20
338/338 [==============================] - 56s 166ms/step - loss: 1.1401 - accuracy: 0.6285 - val_loss: 1.6888 - val_accuracy: 0.3404
Epoch 4/20
338/338 [==============================] - 56s 165ms/step - loss: 1.0200 - accuracy: 0.6712 - val_loss: 1.6530 - val_accuracy: 0.3940
Epoch 5/20
338/338 [==============================] - 56s 167ms/step - loss: 0.9445 - accuracy: 0.6982 - val_loss: 1.6381 - val_accuracy: 0.3865
Epoch 6/20
338/338 [==============================] - 56s 166ms/step - loss: 0.8688 - accuracy: 0.7194 - val_loss: 1.4610 - val_accuracy: 0.4608
Epoch 7/20
338/338 [==============================] - 56s 166ms/step - loss: 0.8249 - accuracy: 0.7340 - val_loss: 1.5530 - val_

In [32]:
print(n)#
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

9
Epoch 1/20
375/375 [==============================] - 60s 151ms/step - loss: 1.8640 - accuracy: 0.3091 - val_loss: 2.3171 - val_accuracy: 0.2831
Epoch 2/20
375/375 [==============================] - 56s 149ms/step - loss: 1.3624 - accuracy: 0.5326 - val_loss: 1.8756 - val_accuracy: 0.3558
Epoch 3/20
375/375 [==============================] - 56s 149ms/step - loss: 1.1609 - accuracy: 0.6101 - val_loss: 1.7181 - val_accuracy: 0.3674
Epoch 4/20
375/375 [==============================] - 56s 148ms/step - loss: 1.0294 - accuracy: 0.6644 - val_loss: 1.5704 - val_accuracy: 0.4132
Epoch 5/20
375/375 [==============================] - 56s 149ms/step - loss: 0.9529 - accuracy: 0.6862 - val_loss: 1.7059 - val_accuracy: 0.4248
Epoch 6/20
375/375 [==============================] - 56s 148ms/step - loss: 0.8863 - accuracy: 0.7078 - val_loss: 1.4688 - val_accuracy: 0.4567
Epoch 7/20
375/375 [==============================] - 56s 149ms/step - loss: 0.8334 - accuracy: 0.7284 - val_loss: 1.4598 - val_

In [33]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

10
Epoch 1/20
413/413 [==============================] - 71s 163ms/step - loss: 1.9042 - accuracy: 0.2876 - val_loss: 2.0614 - val_accuracy: 0.2326
Epoch 2/20
413/413 [==============================] - 67s 163ms/step - loss: 1.3630 - accuracy: 0.5162 - val_loss: 1.9635 - val_accuracy: 0.3251
Epoch 3/20
413/413 [==============================] - 67s 163ms/step - loss: 1.1610 - accuracy: 0.6085 - val_loss: 1.9087 - val_accuracy: 0.3624
Epoch 4/20
413/413 [==============================] - 67s 163ms/step - loss: 1.0097 - accuracy: 0.6606 - val_loss: 1.4870 - val_accuracy: 0.4616
Epoch 5/20
413/413 [==============================] - 67s 163ms/step - loss: 0.9407 - accuracy: 0.6873 - val_loss: 1.4759 - val_accuracy: 0.4680
Epoch 6/20
413/413 [==============================] - 67s 163ms/step - loss: 0.8640 - accuracy: 0.7167 - val_loss: 1.5017 - val_accuracy: 0.4317
Epoch 7/20
413/413 [==============================] - 67s 162ms/step - loss: 0.8340 - accuracy: 0.7245 - val_loss: 1.4645 - val

In [34]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

11
Epoch 1/20
450/450 [==============================] - 77s 162ms/step - loss: 1.9487 - accuracy: 0.2785 - val_loss: 1.9657 - val_accuracy: 0.3332
Epoch 2/20
450/450 [==============================] - 73s 161ms/step - loss: 1.4383 - accuracy: 0.4916 - val_loss: 1.6647 - val_accuracy: 0.4156
Epoch 3/20
450/450 [==============================] - 72s 161ms/step - loss: 1.1944 - accuracy: 0.5933 - val_loss: 1.5389 - val_accuracy: 0.4597
Epoch 4/20
450/450 [==============================] - 72s 161ms/step - loss: 1.0655 - accuracy: 0.6408 - val_loss: 1.5523 - val_accuracy: 0.4460
Epoch 5/20
450/450 [==============================] - 73s 162ms/step - loss: 0.9762 - accuracy: 0.6800 - val_loss: 1.2812 - val_accuracy: 0.5385
Epoch 6/20
450/450 [==============================] - 72s 161ms/step - loss: 0.8907 - accuracy: 0.7037 - val_loss: 1.3191 - val_accuracy: 0.5172
Epoch 7/20
450/450 [==============================] - 72s 161ms/step - loss: 0.8649 - accuracy: 0.7138 - val_loss: 1.2546 - val

In [35]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

12
Epoch 1/20
488/488 [==============================] - 75s 148ms/step - loss: 1.8852 - accuracy: 0.3109 - val_loss: 1.8666 - val_accuracy: 0.3374
Epoch 2/20
488/488 [==============================] - 72s 147ms/step - loss: 1.3516 - accuracy: 0.5279 - val_loss: 1.6713 - val_accuracy: 0.4368
Epoch 3/20
488/488 [==============================] - 72s 147ms/step - loss: 1.1071 - accuracy: 0.6285 - val_loss: 1.4858 - val_accuracy: 0.4816
Epoch 4/20
488/488 [==============================] - 72s 147ms/step - loss: 0.9804 - accuracy: 0.6732 - val_loss: 1.3667 - val_accuracy: 0.5122
Epoch 5/20
488/488 [==============================] - 72s 147ms/step - loss: 0.8950 - accuracy: 0.7087 - val_loss: 1.2212 - val_accuracy: 0.5633
Epoch 6/20
488/488 [==============================] - 72s 147ms/step - loss: 0.8398 - accuracy: 0.7235 - val_loss: 1.4279 - val_accuracy: 0.4963
Epoch 7/20
488/488 [==============================] - 71s 147ms/step - loss: 0.8155 - accuracy: 0.7326 - val_loss: 1.1599 - val

In [36]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

13
Epoch 1/20
525/525 [==============================] - 88s 160ms/step - loss: 1.9018 - accuracy: 0.2921 - val_loss: 1.7854 - val_accuracy: 0.3815
Epoch 2/20
525/525 [==============================] - 84s 160ms/step - loss: 1.3772 - accuracy: 0.5180 - val_loss: 1.5005 - val_accuracy: 0.4771
Epoch 3/20
525/525 [==============================] - 83s 159ms/step - loss: 1.1386 - accuracy: 0.6163 - val_loss: 1.3456 - val_accuracy: 0.5039
Epoch 4/20
525/525 [==============================] - 84s 160ms/step - loss: 1.0107 - accuracy: 0.6614 - val_loss: 1.2778 - val_accuracy: 0.5370
Epoch 5/20
525/525 [==============================] - 84s 159ms/step - loss: 0.9393 - accuracy: 0.6849 - val_loss: 1.2080 - val_accuracy: 0.5721
Epoch 6/20
525/525 [==============================] - 84s 159ms/step - loss: 0.8583 - accuracy: 0.7101 - val_loss: 1.1494 - val_accuracy: 0.5903
Epoch 7/20
525/525 [==============================] - 85s 161ms/step - loss: 0.8129 - accuracy: 0.7265 - val_loss: 1.1883 - val

In [38]:
print(n)#
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

14
Epoch 1/20
563/563 [==============================] - 76s 130ms/step - loss: 1.9371 - accuracy: 0.2857 - val_loss: 1.8075 - val_accuracy: 0.3742
Epoch 2/20
563/563 [==============================] - 73s 129ms/step - loss: 1.3885 - accuracy: 0.5101 - val_loss: 1.4147 - val_accuracy: 0.5006
Epoch 3/20
563/563 [==============================] - 73s 130ms/step - loss: 1.1632 - accuracy: 0.5969 - val_loss: 1.4139 - val_accuracy: 0.5194
Epoch 4/20
563/563 [==============================] - 73s 130ms/step - loss: 1.0436 - accuracy: 0.6440 - val_loss: 1.3000 - val_accuracy: 0.5613
Epoch 5/20
563/563 [==============================] - 73s 129ms/step - loss: 0.9571 - accuracy: 0.6722 - val_loss: 1.2610 - val_accuracy: 0.5431
Epoch 6/20
563/563 [==============================] - 73s 129ms/step - loss: 0.9023 - accuracy: 0.6932 - val_loss: 1.2931 - val_accuracy: 0.5442
Epoch 7/20
563/563 [==============================] - 74s 131ms/step - loss: 0.8464 - accuracy: 0.7112 - val_loss: 1.1360 - val

In [39]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

15
Epoch 1/20
600/600 [==============================] - 100s 159ms/step - loss: 1.9702 - accuracy: 0.2637 - val_loss: 1.7114 - val_accuracy: 0.3982
Epoch 2/20
600/600 [==============================] - 96s 159ms/step - loss: 1.4241 - accuracy: 0.4908 - val_loss: 1.4301 - val_accuracy: 0.4830
Epoch 3/20
600/600 [==============================] - 95s 158ms/step - loss: 1.2205 - accuracy: 0.5752 - val_loss: 1.2961 - val_accuracy: 0.5418
Epoch 4/20
600/600 [==============================] - 95s 158ms/step - loss: 1.0807 - accuracy: 0.6276 - val_loss: 1.2254 - val_accuracy: 0.5677
Epoch 5/20
600/600 [==============================] - 95s 158ms/step - loss: 0.9899 - accuracy: 0.6574 - val_loss: 1.2040 - val_accuracy: 0.5695
Epoch 6/20
600/600 [==============================] - 95s 158ms/step - loss: 0.9288 - accuracy: 0.6823 - val_loss: 1.0218 - val_accuracy: 0.6349
Epoch 7/20
600/600 [==============================] - 94s 157ms/step - loss: 0.8797 - accuracy: 0.7007 - val_loss: 0.9739 - va

In [40]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

16
Epoch 1/20
638/638 [==============================] - 96s 146ms/step - loss: 1.9978 - accuracy: 0.2583 - val_loss: 1.7086 - val_accuracy: 0.3941
Epoch 2/20
638/638 [==============================] - 94s 147ms/step - loss: 1.4508 - accuracy: 0.4763 - val_loss: 1.4177 - val_accuracy: 0.5039
Epoch 3/20
638/638 [==============================] - 93s 146ms/step - loss: 1.2238 - accuracy: 0.5668 - val_loss: 1.3481 - val_accuracy: 0.5137
Epoch 4/20
638/638 [==============================] - 95s 148ms/step - loss: 1.0821 - accuracy: 0.6224 - val_loss: 1.0749 - val_accuracy: 0.6165
Epoch 5/20
638/638 [==============================] - 102s 160ms/step - loss: 1.0055 - accuracy: 0.6507 - val_loss: 1.0678 - val_accuracy: 0.6200
Epoch 6/20
638/638 [==============================] - 103s 161ms/step - loss: 0.9314 - accuracy: 0.6796 - val_loss: 0.9764 - val_accuracy: 0.6615
Epoch 7/20
638/638 [==============================] - 102s 161ms/step - loss: 0.8909 - accuracy: 0.6925 - val_loss: 1.0024 - 

In [41]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

17
Epoch 1/20
675/675 [==============================] - 102s 147ms/step - loss: 1.9858 - accuracy: 0.2609 - val_loss: 1.5826 - val_accuracy: 0.4372
Epoch 2/20
675/675 [==============================] - 99s 146ms/step - loss: 1.4242 - accuracy: 0.4861 - val_loss: 1.2867 - val_accuracy: 0.5433
Epoch 3/20
675/675 [==============================] - 98s 145ms/step - loss: 1.2156 - accuracy: 0.5705 - val_loss: 1.3652 - val_accuracy: 0.5265
Epoch 4/20
675/675 [==============================] - 97s 144ms/step - loss: 1.0910 - accuracy: 0.6192 - val_loss: 1.0828 - val_accuracy: 0.6256
Epoch 5/20
675/675 [==============================] - 97s 144ms/step - loss: 1.0085 - accuracy: 0.6493 - val_loss: 1.0207 - val_accuracy: 0.6572
Epoch 6/20
675/675 [==============================] - 97s 144ms/step - loss: 0.9332 - accuracy: 0.6765 - val_loss: 0.9415 - val_accuracy: 0.6722
Epoch 7/20
675/675 [==============================] - 97s 144ms/step - loss: 0.8945 - accuracy: 0.6888 - val_loss: 0.8870 - va

In [42]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

18
Epoch 1/20
713/713 [==============================] - 107s 146ms/step - loss: 2.0242 - accuracy: 0.2500 - val_loss: 1.6671 - val_accuracy: 0.3968
Epoch 2/20
713/713 [==============================] - 103s 145ms/step - loss: 1.4848 - accuracy: 0.4672 - val_loss: 1.2789 - val_accuracy: 0.5518
Epoch 3/20
713/713 [==============================] - 103s 145ms/step - loss: 1.2350 - accuracy: 0.5616 - val_loss: 1.0640 - val_accuracy: 0.6343
Epoch 4/20
713/713 [==============================] - 103s 144ms/step - loss: 1.1096 - accuracy: 0.6087 - val_loss: 0.9917 - val_accuracy: 0.6614
Epoch 5/20
713/713 [==============================] - 103s 144ms/step - loss: 1.0417 - accuracy: 0.6351 - val_loss: 0.9209 - val_accuracy: 0.6790
Epoch 6/20
713/713 [==============================] - 103s 144ms/step - loss: 0.9726 - accuracy: 0.6615 - val_loss: 0.8966 - val_accuracy: 0.6957
Epoch 7/20
713/713 [==============================] - 102s 144ms/step - loss: 0.9229 - accuracy: 0.6776 - val_loss: 0.886

In [43]:
print(n)#
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

19
Epoch 1/20
750/750 [==============================] - 111s 144ms/step - loss: 1.9887 - accuracy: 0.2617 - val_loss: 1.4835 - val_accuracy: 0.4675
Epoch 2/20
750/750 [==============================] - 108s 144ms/step - loss: 1.4679 - accuracy: 0.4733 - val_loss: 1.1515 - val_accuracy: 0.6077
Epoch 3/20
750/750 [==============================] - 107s 143ms/step - loss: 1.2430 - accuracy: 0.5612 - val_loss: 1.0124 - val_accuracy: 0.6475
Epoch 4/20
750/750 [==============================] - 111s 148ms/step - loss: 1.1042 - accuracy: 0.6162 - val_loss: 1.0058 - val_accuracy: 0.6569
Epoch 5/20
750/750 [==============================] - 108s 144ms/step - loss: 1.0236 - accuracy: 0.6451 - val_loss: 0.9312 - val_accuracy: 0.6732
Epoch 6/20
750/750 [==============================] - 110s 146ms/step - loss: 0.9501 - accuracy: 0.6679 - val_loss: 0.8198 - val_accuracy: 0.7118
Epoch 7/20
750/750 [==============================] - 109s 145ms/step - loss: 0.9059 - accuracy: 0.6845 - val_loss: 0.839

In [ ]:
time.sleep(400)

In [ ]:
len(models_lsa)

In [44]:
#new_model_lsa_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_lsa_e2"
new_model_lsa_dir  = "D:/models/aug_22/"+dataset+"/C1/"+dataset+"_model_c1_aug_lsa_e1"

i=14

for model in models_lsa[14:]:
    model.save(new_model_lsa_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_lsa_e1_14\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_lsa_e1_15\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_lsa_e1_16\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_lsa_e1_17\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_lsa_e1_18\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_lsa_e1_19\assets
Model has been saved


In [45]:
import gc

In [46]:
try:
    del lsa_values
    del top_images_by_lsa
    del top_labels_by_lsa
    del image_sets_lsa
    del label_sets_lsa
    del models_lsa
except:
    print("Error")

In [47]:
gc.collect()

61172

In [ ]:
loading = False

models_lsa = []

if loading:
    for i in range(10,20):
        #model_lsa_dir = "D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_lsa_e1_"+str(i)
        print(model_lsa_dir)
        model =utils.My_model(dataset,True,model_lsa_dir)
        model.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        models_lsa.append(model)
       
    

## Training guided by DSA

In [48]:
dsa_direction = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/dsa_values.npy"

In [49]:
dsa_values = np.load(dsa_direction)

In [50]:
top_images_by_dsa = utils.get_x_of_indexes(list(np.flip(np.argsort(dsa_values))),x_train_and_adversary)
top_labels_by_dsa = utils.get_x_of_indexes(list(np.flip(np.argsort(dsa_values))),y_train_and_adversary)

In [51]:
m = n_data_points
n = 0
image_sets_dsa = []
label_sets_dsa = []


for i in range((len(top_images_by_dsa)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_dsa)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_dsa)%m))
        top_images_by_dsa_n = np.array(top_images_by_dsa[:n+m+(len(top_images_by_dsa)%m)])
        top_labels_by_dsa_n = np.array(top_labels_by_dsa[:n+m+(len(top_images_by_dsa)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_dsa_n = np.array(top_images_by_dsa[:n+m])
        top_labels_by_dsa_n = np.array(top_labels_by_dsa[:n+m])
    image_sets_dsa.append(top_images_by_dsa_n)
    label_sets_dsa.append(top_labels_by_dsa_n)
    print(len(top_images_by_dsa_n))
    n += m



0 :
0  ->  2400
2400
1 :
0  ->  4800
4800
2 :
0  ->  7200
7200
3 :
0  ->  9600
9600
4 :
0  ->  12000
12000
5 :
0  ->  14400
14400
6 :
0  ->  16800
16800
7 :
0  ->  19200
19200
8 :
0  ->  21600
21600
9 :
0  ->  24000
24000
10 :
0  ->  26400
26400
11 :
0  ->  28800
28800
12 :
0  ->  31200
31200
13 :
0  ->  33600
33600
14 :
0  ->  36000
36000
15 :
0  ->  38400
38400
16 :
0  ->  40800
40800
17 :
0  ->  43200
43200
18 :
0  ->  45600
45600
19 :
Last
0  ->  50399
47999


In [52]:
print(model_dir)

models_dsa = []
for i in range(len(label_sets_dsa)):
    print(i,":")
    model = utils.My_model(dataset,False,None)
    model.compile_model()
    models_dsa.append(model)


C:/Users/fjdur/Documents/upc-july/models/model_02
0 :
Model compiled
1 :
Model compiled
2 :
Model compiled
3 :
Model compiled
4 :
Model compiled
5 :
Model compiled
6 :
Model compiled
7 :
Model compiled
8 :
Model compiled
9 :
Model compiled
10 :
Model compiled
11 :
Model compiled
12 :
Model compiled
13 :
Model compiled
14 :
Model compiled
15 :
Model compiled
16 :
Model compiled
17 :
Model compiled
18 :
Model compiled
19 :
Model compiled


In [53]:
n=0
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

0
Epoch 1/20
38/38 [==============================] - 16s 316ms/step - loss: 2.3019 - accuracy: 0.1195 - val_loss: 2.2961 - val_accuracy: 0.1458
Epoch 2/20
38/38 [==============================] - 11s 304ms/step - loss: 2.2647 - accuracy: 0.1476 - val_loss: 2.1190 - val_accuracy: 0.2256
Epoch 3/20
38/38 [==============================] - 13s 334ms/step - loss: 2.1632 - accuracy: 0.2284 - val_loss: 2.1282 - val_accuracy: 0.2324
Epoch 4/20
38/38 [==============================] - 12s 333ms/step - loss: 2.0683 - accuracy: 0.2369 - val_loss: 2.0281 - val_accuracy: 0.2788
Epoch 5/20
38/38 [==============================] - 12s 333ms/step - loss: 1.9686 - accuracy: 0.2845 - val_loss: 2.0966 - val_accuracy: 0.2542
Epoch 6/20
38/38 [==============================] - 13s 334ms/step - loss: 1.8905 - accuracy: 0.3232 - val_loss: 1.9320 - val_accuracy: 0.3057
Epoch 7/20
38/38 [==============================] - 12s 332ms/step - loss: 1.7881 - accuracy: 0.3646 - val_loss: 2.0239 - val_accuracy: 0.27

In [54]:
n=1
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

1
Epoch 1/20
75/75 [==============================] - 23s 250ms/step - loss: 2.3010 - accuracy: 0.1146 - val_loss: 2.1891 - val_accuracy: 0.1982
Epoch 2/20
75/75 [==============================] - 18s 243ms/step - loss: 2.1493 - accuracy: 0.2008 - val_loss: 1.9902 - val_accuracy: 0.2694
Epoch 3/20
75/75 [==============================] - 18s 242ms/step - loss: 2.0130 - accuracy: 0.2567 - val_loss: 1.8730 - val_accuracy: 0.3231
Epoch 4/20
75/75 [==============================] - 18s 241ms/step - loss: 1.9088 - accuracy: 0.3087 - val_loss: 1.8148 - val_accuracy: 0.3372
Epoch 5/20
75/75 [==============================] - 18s 242ms/step - loss: 1.8037 - accuracy: 0.3385 - val_loss: 1.7322 - val_accuracy: 0.3738
Epoch 6/20
75/75 [==============================] - 18s 240ms/step - loss: 1.7210 - accuracy: 0.3906 - val_loss: 1.6696 - val_accuracy: 0.3923
Epoch 7/20
75/75 [==============================] - 18s 241ms/step - loss: 1.6564 - accuracy: 0.4056 - val_loss: 1.6268 - val_accuracy: 0.40

In [55]:
n=2
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

2
Epoch 1/20
113/113 [==============================] - 28s 209ms/step - loss: 2.2740 - accuracy: 0.1268 - val_loss: 2.1280 - val_accuracy: 0.2376
Epoch 2/20
113/113 [==============================] - 24s 211ms/step - loss: 2.0482 - accuracy: 0.2504 - val_loss: 1.8445 - val_accuracy: 0.3109
Epoch 3/20
113/113 [==============================] - 24s 213ms/step - loss: 1.8750 - accuracy: 0.3090 - val_loss: 1.8907 - val_accuracy: 0.3318
Epoch 4/20
113/113 [==============================] - 25s 218ms/step - loss: 1.7414 - accuracy: 0.3622 - val_loss: 1.6495 - val_accuracy: 0.3981
Epoch 5/20
113/113 [==============================] - 24s 210ms/step - loss: 1.6286 - accuracy: 0.4057 - val_loss: 1.6337 - val_accuracy: 0.4098
Epoch 6/20
113/113 [==============================] - 24s 210ms/step - loss: 1.5404 - accuracy: 0.4378 - val_loss: 1.4498 - val_accuracy: 0.4837
Epoch 7/20
113/113 [==============================] - 24s 210ms/step - loss: 1.4428 - accuracy: 0.4841 - val_loss: 1.4739 - val_

In [56]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

3
Epoch 1/20
150/150 [==============================] - 33s 184ms/step - loss: 2.2372 - accuracy: 0.1397 - val_loss: 1.8346 - val_accuracy: 0.3237
Epoch 2/20
150/150 [==============================] - 26s 176ms/step - loss: 1.9098 - accuracy: 0.2837 - val_loss: 1.7054 - val_accuracy: 0.3877
Epoch 3/20
150/150 [==============================] - 26s 176ms/step - loss: 1.7613 - accuracy: 0.3441 - val_loss: 1.6174 - val_accuracy: 0.4197
Epoch 4/20
150/150 [==============================] - 26s 175ms/step - loss: 1.6655 - accuracy: 0.3915 - val_loss: 1.5331 - val_accuracy: 0.4436
Epoch 5/20
150/150 [==============================] - 26s 176ms/step - loss: 1.5596 - accuracy: 0.4336 - val_loss: 1.4744 - val_accuracy: 0.4713
Epoch 6/20
150/150 [==============================] - 26s 175ms/step - loss: 1.4904 - accuracy: 0.4727 - val_loss: 1.4820 - val_accuracy: 0.4711
Epoch 7/20
150/150 [==============================] - 26s 177ms/step - loss: 1.4038 - accuracy: 0.5032 - val_loss: 1.4836 - val_

In [57]:
print(n)#

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

4
Epoch 1/20
188/188 [==============================] - 35s 169ms/step - loss: 2.2465 - accuracy: 0.1492 - val_loss: 1.9355 - val_accuracy: 0.2862
Epoch 2/20
188/188 [==============================] - 32s 169ms/step - loss: 1.9141 - accuracy: 0.3013 - val_loss: 1.7008 - val_accuracy: 0.3677
Epoch 3/20
188/188 [==============================] - 32s 169ms/step - loss: 1.7093 - accuracy: 0.3775 - val_loss: 1.6462 - val_accuracy: 0.3995
Epoch 4/20
188/188 [==============================] - 32s 169ms/step - loss: 1.5696 - accuracy: 0.4368 - val_loss: 1.4810 - val_accuracy: 0.4748
Epoch 5/20
188/188 [==============================] - 32s 170ms/step - loss: 1.4845 - accuracy: 0.4686 - val_loss: 1.4098 - val_accuracy: 0.4883
Epoch 6/20
188/188 [==============================] - 32s 169ms/step - loss: 1.3868 - accuracy: 0.5039 - val_loss: 1.3684 - val_accuracy: 0.5166
Epoch 7/20
188/188 [==============================] - 32s 170ms/step - loss: 1.3234 - accuracy: 0.5308 - val_loss: 1.3449 - val_

In [58]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

5
Epoch 1/20
225/225 [==============================] - 36s 147ms/step - loss: 2.2270 - accuracy: 0.1603 - val_loss: 1.8233 - val_accuracy: 0.3470
Epoch 2/20
225/225 [==============================] - 32s 143ms/step - loss: 1.8645 - accuracy: 0.3187 - val_loss: 1.7136 - val_accuracy: 0.3708
Epoch 3/20
225/225 [==============================] - 32s 143ms/step - loss: 1.6831 - accuracy: 0.3901 - val_loss: 1.5128 - val_accuracy: 0.4476
Epoch 4/20
225/225 [==============================] - 32s 144ms/step - loss: 1.5518 - accuracy: 0.4377 - val_loss: 1.4318 - val_accuracy: 0.4818
Epoch 5/20
225/225 [==============================] - 33s 146ms/step - loss: 1.4370 - accuracy: 0.4848 - val_loss: 1.3188 - val_accuracy: 0.5390
Epoch 6/20
225/225 [==============================] - 33s 146ms/step - loss: 1.3548 - accuracy: 0.5196 - val_loss: 1.3186 - val_accuracy: 0.5412
Epoch 7/20
225/225 [==============================] - 34s 149ms/step - loss: 1.2693 - accuracy: 0.5451 - val_loss: 1.2267 - val_

In [59]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

6
Epoch 1/20
263/263 [==============================] - 45s 160ms/step - loss: 2.1864 - accuracy: 0.1747 - val_loss: 1.8383 - val_accuracy: 0.3339
Epoch 2/20
263/263 [==============================] - 42s 159ms/step - loss: 1.7784 - accuracy: 0.3465 - val_loss: 1.8208 - val_accuracy: 0.3664
Epoch 3/20
263/263 [==============================] - 42s 160ms/step - loss: 1.6021 - accuracy: 0.4180 - val_loss: 1.4415 - val_accuracy: 0.4721
Epoch 4/20
263/263 [==============================] - 42s 160ms/step - loss: 1.4595 - accuracy: 0.4818 - val_loss: 1.5561 - val_accuracy: 0.4457
Epoch 5/20
263/263 [==============================] - 42s 159ms/step - loss: 1.3435 - accuracy: 0.5163 - val_loss: 1.3818 - val_accuracy: 0.5085
Epoch 6/20
263/263 [==============================] - 42s 159ms/step - loss: 1.2592 - accuracy: 0.5500 - val_loss: 1.2194 - val_accuracy: 0.5724
Epoch 7/20
263/263 [==============================] - 42s 160ms/step - loss: 1.1733 - accuracy: 0.5857 - val_loss: 1.1566 - val_

In [60]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

7
Epoch 1/20
300/300 [==============================] - 53s 163ms/step - loss: 2.1843 - accuracy: 0.1790 - val_loss: 1.7054 - val_accuracy: 0.3769
Epoch 2/20
300/300 [==============================] - 47s 156ms/step - loss: 1.7748 - accuracy: 0.3566 - val_loss: 1.5107 - val_accuracy: 0.4547
Epoch 3/20
300/300 [==============================] - 47s 156ms/step - loss: 1.5773 - accuracy: 0.4259 - val_loss: 1.4855 - val_accuracy: 0.4655
Epoch 4/20
300/300 [==============================] - 47s 156ms/step - loss: 1.4446 - accuracy: 0.4811 - val_loss: 1.3059 - val_accuracy: 0.5472
Epoch 5/20
300/300 [==============================] - 47s 155ms/step - loss: 1.3148 - accuracy: 0.5328 - val_loss: 1.1612 - val_accuracy: 0.5893
Epoch 6/20
300/300 [==============================] - 47s 158ms/step - loss: 1.2019 - accuracy: 0.5725 - val_loss: 1.1454 - val_accuracy: 0.5931
Epoch 7/20
300/300 [==============================] - 47s 156ms/step - loss: 1.1411 - accuracy: 0.5970 - val_loss: 1.0949 - val_

In [61]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

8
Epoch 1/20
338/338 [==============================] - 56s 155ms/step - loss: 2.1653 - accuracy: 0.1836 - val_loss: 1.7246 - val_accuracy: 0.3581
Epoch 2/20
338/338 [==============================] - 52s 155ms/step - loss: 1.7174 - accuracy: 0.3842 - val_loss: 1.4747 - val_accuracy: 0.4630
Epoch 3/20
338/338 [==============================] - 52s 155ms/step - loss: 1.4968 - accuracy: 0.4603 - val_loss: 1.3640 - val_accuracy: 0.5197
Epoch 4/20
338/338 [==============================] - 52s 155ms/step - loss: 1.3655 - accuracy: 0.5109 - val_loss: 1.3175 - val_accuracy: 0.5426
Epoch 5/20
338/338 [==============================] - 52s 155ms/step - loss: 1.2484 - accuracy: 0.5625 - val_loss: 1.1896 - val_accuracy: 0.5767
Epoch 6/20
338/338 [==============================] - 52s 155ms/step - loss: 1.1716 - accuracy: 0.5805 - val_loss: 1.1788 - val_accuracy: 0.5944
Epoch 7/20
338/338 [==============================] - 52s 154ms/step - loss: 1.0957 - accuracy: 0.6103 - val_loss: 1.0933 - val_

In [62]:
print(n)#

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

9
Epoch 1/20
375/375 [==============================] - 67s 168ms/step - loss: 2.1462 - accuracy: 0.2015 - val_loss: 1.6658 - val_accuracy: 0.4040
Epoch 2/20
375/375 [==============================] - 63s 167ms/step - loss: 1.7131 - accuracy: 0.3811 - val_loss: 1.4814 - val_accuracy: 0.4703
Epoch 3/20
375/375 [==============================] - 63s 167ms/step - loss: 1.5095 - accuracy: 0.4609 - val_loss: 1.3858 - val_accuracy: 0.5098
Epoch 4/20
375/375 [==============================] - 64s 170ms/step - loss: 1.3790 - accuracy: 0.5105 - val_loss: 1.2147 - val_accuracy: 0.5716
Epoch 5/20
375/375 [==============================] - 57s 153ms/step - loss: 1.2572 - accuracy: 0.5543 - val_loss: 1.1457 - val_accuracy: 0.5956
Epoch 6/20
375/375 [==============================] - 57s 153ms/step - loss: 1.1705 - accuracy: 0.5863 - val_loss: 1.0555 - val_accuracy: 0.6309
Epoch 7/20
375/375 [==============================] - 58s 154ms/step - loss: 1.1222 - accuracy: 0.6048 - val_loss: 1.0120 - val_

In [63]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

10
Epoch 1/20
413/413 [==============================] - 65s 151ms/step - loss: 2.0972 - accuracy: 0.2148 - val_loss: 1.6839 - val_accuracy: 0.3939
Epoch 2/20
413/413 [==============================] - 62s 151ms/step - loss: 1.6693 - accuracy: 0.3871 - val_loss: 1.4436 - val_accuracy: 0.4798
Epoch 3/20
413/413 [==============================] - 63s 152ms/step - loss: 1.4747 - accuracy: 0.4705 - val_loss: 1.3225 - val_accuracy: 0.5272
Epoch 4/20
413/413 [==============================] - 63s 152ms/step - loss: 1.3252 - accuracy: 0.5302 - val_loss: 1.2364 - val_accuracy: 0.5621
Epoch 5/20
413/413 [==============================] - 63s 152ms/step - loss: 1.2230 - accuracy: 0.5625 - val_loss: 1.2559 - val_accuracy: 0.5608
Epoch 6/20
413/413 [==============================] - 62s 151ms/step - loss: 1.1323 - accuracy: 0.6060 - val_loss: 1.0894 - val_accuracy: 0.6173
Epoch 7/20
413/413 [==============================] - 62s 151ms/step - loss: 1.0610 - accuracy: 0.6301 - val_loss: 0.9867 - val

In [64]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

11
Epoch 1/20
450/450 [==============================] - 71s 150ms/step - loss: 2.1092 - accuracy: 0.2111 - val_loss: 1.6200 - val_accuracy: 0.4036
Epoch 2/20
450/450 [==============================] - 67s 149ms/step - loss: 1.6429 - accuracy: 0.4080 - val_loss: 1.5463 - val_accuracy: 0.4393
Epoch 3/20
450/450 [==============================] - 68s 151ms/step - loss: 1.4289 - accuracy: 0.4915 - val_loss: 1.2402 - val_accuracy: 0.5616
Epoch 4/20
450/450 [==============================] - 67s 149ms/step - loss: 1.2734 - accuracy: 0.5530 - val_loss: 1.1970 - val_accuracy: 0.5778
Epoch 5/20
450/450 [==============================] - 67s 149ms/step - loss: 1.1807 - accuracy: 0.5857 - val_loss: 1.0430 - val_accuracy: 0.6399
Epoch 6/20
450/450 [==============================] - 67s 149ms/step - loss: 1.0897 - accuracy: 0.6199 - val_loss: 1.0553 - val_accuracy: 0.6321
Epoch 7/20
450/450 [==============================] - 67s 149ms/step - loss: 1.0390 - accuracy: 0.6393 - val_loss: 0.9089 - val

In [65]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

12
Epoch 1/20
488/488 [==============================] - 77s 151ms/step - loss: 2.0999 - accuracy: 0.2087 - val_loss: 1.6162 - val_accuracy: 0.4236
Epoch 2/20
488/488 [==============================] - 81s 165ms/step - loss: 1.6200 - accuracy: 0.4153 - val_loss: 1.4155 - val_accuracy: 0.4839
Epoch 3/20
488/488 [==============================] - 81s 167ms/step - loss: 1.3882 - accuracy: 0.5072 - val_loss: 1.2456 - val_accuracy: 0.5524
Epoch 4/20
488/488 [==============================] - 81s 166ms/step - loss: 1.2380 - accuracy: 0.5628 - val_loss: 1.0987 - val_accuracy: 0.6134
Epoch 5/20
488/488 [==============================] - 81s 165ms/step - loss: 1.1481 - accuracy: 0.5972 - val_loss: 1.0211 - val_accuracy: 0.6506
Epoch 6/20
488/488 [==============================] - 81s 166ms/step - loss: 1.0703 - accuracy: 0.6300 - val_loss: 0.9620 - val_accuracy: 0.6611
Epoch 7/20
488/488 [==============================] - 81s 166ms/step - loss: 0.9955 - accuracy: 0.6529 - val_loss: 0.9268 - val

In [66]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

13
Epoch 1/20
525/525 [==============================] - 90s 164ms/step - loss: 2.0754 - accuracy: 0.2224 - val_loss: 1.6449 - val_accuracy: 0.4215
Epoch 2/20
525/525 [==============================] - 90s 171ms/step - loss: 1.6023 - accuracy: 0.4235 - val_loss: 1.3563 - val_accuracy: 0.5293
Epoch 3/20
525/525 [==============================] - 86s 164ms/step - loss: 1.3820 - accuracy: 0.5073 - val_loss: 1.2067 - val_accuracy: 0.5715
Epoch 4/20
525/525 [==============================] - 86s 164ms/step - loss: 1.2482 - accuracy: 0.5660 - val_loss: 1.0627 - val_accuracy: 0.6248
Epoch 5/20
525/525 [==============================] - 85s 162ms/step - loss: 1.1437 - accuracy: 0.6011 - val_loss: 1.0378 - val_accuracy: 0.6381
Epoch 6/20
525/525 [==============================] - 77s 147ms/step - loss: 1.0757 - accuracy: 0.6180 - val_loss: 0.9402 - val_accuracy: 0.6702
Epoch 7/20
525/525 [==============================] - 77s 146ms/step - loss: 1.0185 - accuracy: 0.6446 - val_loss: 0.8946 - val

In [67]:
print(n)#

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

14
Epoch 1/20
563/563 [==============================] - 86s 147ms/step - loss: 2.0699 - accuracy: 0.2214 - val_loss: 1.5435 - val_accuracy: 0.4353
Epoch 2/20
563/563 [==============================] - 82s 146ms/step - loss: 1.5331 - accuracy: 0.4486 - val_loss: 1.2747 - val_accuracy: 0.5483
Epoch 3/20
563/563 [==============================] - 83s 147ms/step - loss: 1.3188 - accuracy: 0.5313 - val_loss: 1.2148 - val_accuracy: 0.5715
Epoch 4/20
563/563 [==============================] - 84s 150ms/step - loss: 1.1808 - accuracy: 0.5872 - val_loss: 1.0520 - val_accuracy: 0.6388
Epoch 5/20
563/563 [==============================] - 92s 163ms/step - loss: 1.0993 - accuracy: 0.6170 - val_loss: 0.9826 - val_accuracy: 0.6582
Epoch 6/20
563/563 [==============================] - 91s 161ms/step - loss: 1.0152 - accuracy: 0.6459 - val_loss: 0.9476 - val_accuracy: 0.6846
Epoch 7/20
563/563 [==============================] - 91s 162ms/step - loss: 0.9658 - accuracy: 0.6660 - val_loss: 0.8721 - val

In [68]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

15
Epoch 1/20
600/600 [==============================] - 102s 161ms/step - loss: 2.0487 - accuracy: 0.2404 - val_loss: 1.5369 - val_accuracy: 0.4426
Epoch 2/20
600/600 [==============================] - 96s 160ms/step - loss: 1.5121 - accuracy: 0.4578 - val_loss: 1.2315 - val_accuracy: 0.5699
Epoch 3/20
600/600 [==============================] - 96s 160ms/step - loss: 1.3170 - accuracy: 0.5368 - val_loss: 1.1450 - val_accuracy: 0.5960
Epoch 4/20
600/600 [==============================] - 96s 160ms/step - loss: 1.1905 - accuracy: 0.5861 - val_loss: 0.9890 - val_accuracy: 0.6575
Epoch 5/20
600/600 [==============================] - 96s 160ms/step - loss: 1.0805 - accuracy: 0.6234 - val_loss: 0.9843 - val_accuracy: 0.6601
Epoch 6/20
600/600 [==============================] - 96s 160ms/step - loss: 1.0141 - accuracy: 0.6498 - val_loss: 0.9199 - val_accuracy: 0.6859
Epoch 7/20
600/600 [==============================] - 96s 160ms/step - loss: 0.9543 - accuracy: 0.6687 - val_loss: 0.8478 - va

In [69]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

16
Epoch 1/20
638/638 [==============================] - 107s 161ms/step - loss: 2.0272 - accuracy: 0.2423 - val_loss: 1.5109 - val_accuracy: 0.4554
Epoch 2/20
638/638 [==============================] - 103s 161ms/step - loss: 1.5296 - accuracy: 0.4534 - val_loss: 1.2852 - val_accuracy: 0.5555
Epoch 3/20
638/638 [==============================] - 103s 161ms/step - loss: 1.3000 - accuracy: 0.5420 - val_loss: 1.1488 - val_accuracy: 0.5924
Epoch 4/20
638/638 [==============================] - 103s 162ms/step - loss: 1.1528 - accuracy: 0.5959 - val_loss: 1.0596 - val_accuracy: 0.6324
Epoch 5/20
638/638 [==============================] - 103s 162ms/step - loss: 1.0604 - accuracy: 0.6290 - val_loss: 1.0084 - val_accuracy: 0.6410
Epoch 6/20
638/638 [==============================] - 103s 161ms/step - loss: 1.0040 - accuracy: 0.6580 - val_loss: 0.9625 - val_accuracy: 0.6800
Epoch 7/20
638/638 [==============================] - 103s 161ms/step - loss: 0.9400 - accuracy: 0.6761 - val_loss: 0.888

In [70]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

17
Epoch 1/20
675/675 [==============================] - 111s 155ms/step - loss: 2.0467 - accuracy: 0.2408 - val_loss: 1.6091 - val_accuracy: 0.4096
Epoch 2/20
675/675 [==============================] - 104s 154ms/step - loss: 1.5144 - accuracy: 0.4542 - val_loss: 1.2391 - val_accuracy: 0.5718
Epoch 3/20
675/675 [==============================] - 104s 153ms/step - loss: 1.2923 - accuracy: 0.5426 - val_loss: 1.0610 - val_accuracy: 0.6277
Epoch 4/20
675/675 [==============================] - 105s 156ms/step - loss: 1.1411 - accuracy: 0.6009 - val_loss: 1.0300 - val_accuracy: 0.6398
Epoch 5/20
675/675 [==============================] - 104s 154ms/step - loss: 1.0545 - accuracy: 0.6382 - val_loss: 0.9743 - val_accuracy: 0.6543
Epoch 6/20
675/675 [==============================] - 104s 155ms/step - loss: 0.9886 - accuracy: 0.6591 - val_loss: 0.9208 - val_accuracy: 0.6923
Epoch 7/20
675/675 [==============================] - 108s 160ms/step - loss: 0.9403 - accuracy: 0.6714 - val_loss: 0.826

In [71]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

18
Epoch 1/20
713/713 [==============================] - 119s 160ms/step - loss: 2.0060 - accuracy: 0.2542 - val_loss: 1.5129 - val_accuracy: 0.4434
Epoch 2/20
713/713 [==============================] - 114s 160ms/step - loss: 1.4542 - accuracy: 0.4756 - val_loss: 1.1655 - val_accuracy: 0.5936
Epoch 3/20
713/713 [==============================] - 114s 161ms/step - loss: 1.2263 - accuracy: 0.5714 - val_loss: 1.1093 - val_accuracy: 0.6087
Epoch 4/20
713/713 [==============================] - 114s 160ms/step - loss: 1.0962 - accuracy: 0.6190 - val_loss: 1.0489 - val_accuracy: 0.6428
Epoch 5/20
713/713 [==============================] - 114s 160ms/step - loss: 1.0170 - accuracy: 0.6470 - val_loss: 0.8814 - val_accuracy: 0.6938
Epoch 6/20
713/713 [==============================] - 115s 161ms/step - loss: 0.9619 - accuracy: 0.6668 - val_loss: 0.9275 - val_accuracy: 0.6833
Epoch 7/20
713/713 [==============================] - 115s 161ms/step - loss: 0.9127 - accuracy: 0.6837 - val_loss: 0.823

In [72]:
time.sleep(400)

In [73]:
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

19
Epoch 1/20
750/750 [==============================] - 119s 151ms/step - loss: 1.9925 - accuracy: 0.2620 - val_loss: 1.4086 - val_accuracy: 0.5008
Epoch 2/20
750/750 [==============================] - 111s 148ms/step - loss: 1.4277 - accuracy: 0.4922 - val_loss: 1.1944 - val_accuracy: 0.5817
Epoch 3/20
750/750 [==============================] - 111s 149ms/step - loss: 1.2186 - accuracy: 0.5722 - val_loss: 1.0924 - val_accuracy: 0.6223
Epoch 4/20
750/750 [==============================] - 110s 147ms/step - loss: 1.0943 - accuracy: 0.6230 - val_loss: 0.9248 - val_accuracy: 0.6824
Epoch 5/20
750/750 [==============================] - 112s 149ms/step - loss: 1.0202 - accuracy: 0.6457 - val_loss: 0.9577 - val_accuracy: 0.6726
Epoch 6/20
750/750 [==============================] - 114s 152ms/step - loss: 0.9616 - accuracy: 0.6642 - val_loss: 0.8443 - val_accuracy: 0.7133
Epoch 7/20
750/750 [==============================] - 120s 160ms/step - loss: 0.9025 - accuracy: 0.6879 - val_loss: 0.802

In [ ]:
time.sleep(400)

In [74]:
new_model_dsa_dir  = "D:/models/aug_22/"+dataset+"/C1/"+dataset+"_model_c1_aug_dsa_e1"

i=0

for model in models_dsa:
    model.save(new_model_dsa_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_dsa_e1_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_dsa_e1_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_dsa_e1_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_dsa_e1_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_dsa_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_dsa_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_dsa_e1_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_dsa_e1_7\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cif

In [ ]:
loading = False

models_dsa = []

if loading:
    for i in range(20):
        model_dsa_dir = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_dsa_e2_"+str(i)
        print(model_dsa_dir)
        model =utils.My_model('gtsrb',True,model_dsa_dir)
        model.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        models_dsa.append(model)
       
    

In [75]:
try:
    del dsa_values
    del top_images_by_dsa
    del top_labels_by_dsa
    del image_sets_dsa
    del label_sets_dsa
    del models_dsa
except:
    print("Error")

In [76]:
gc.collect()

61157

## Uncertainty

In [16]:
dg_direction = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/deep_gini_values.npy"

deep_gini_values = np.load(dg_direction)


In [17]:
# Obtaining top n images by dg values
top_images_by_dg  = utils.get_x_of_indexes(list(np.flip(np.argsort(deep_gini_values))),x_train_and_adversary)
top_labels_by_dg = utils.get_x_of_indexes(list(np.flip(np.argsort(deep_gini_values))),y_train_and_adversary)

In [18]:
m = n_data_points
n = 0
image_sets_dg = []
label_sets_dg = []

# last
#for i in range(0,len(top_images_by_lsa)//m):

for i in range((len(top_images_by_dg)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_dg)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_dg)%m))
        top_images_by_dg_n = np.array(top_images_by_dg[:n+m+(len(top_images_by_dg)%m)])
        top_labels_by_dg_n = np.array(top_labels_by_dg[:n+m+(len(top_images_by_dg)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_dg_n = np.array(top_images_by_dg[:n+m])
        top_labels_by_dg_n = np.array(top_labels_by_dg[:n+m])
    image_sets_dg.append(top_images_by_dg_n)
    label_sets_dg.append(top_labels_by_dg_n)
    print(len(top_images_by_dg_n))
    n += m



0 :
0  ->  2400
2400
1 :
0  ->  4800
4800
2 :
0  ->  7200
7200
3 :
0  ->  9600
9600
4 :
0  ->  12000
12000
5 :
0  ->  14400
14400
6 :
0  ->  16800
16800
7 :
0  ->  19200
19200
8 :
0  ->  21600
21600
9 :
0  ->  24000
24000
10 :
0  ->  26400
26400
11 :
0  ->  28800
28800
12 :
0  ->  31200
31200
13 :
0  ->  33600
33600
14 :
0  ->  36000
36000
15 :
0  ->  38400
38400
16 :
0  ->  40800
40800
17 :
0  ->  43200
43200
18 :
0  ->  45600
45600
19 :
Last
0  ->  50399
47999


In [20]:
models_dg = []
#for i in range(len(label_sets_lsa)):
for i in range(len(label_sets_dg)):
    print(i,":")
    model = utils.My_model(dataset,False)
    model.compile_model()
    models_dg.append(model)

0 :
Model compiled
1 :
Model compiled
2 :
Model compiled
3 :
Model compiled
4 :
Model compiled
5 :
Model compiled
6 :
Model compiled
7 :
Model compiled
8 :
Model compiled
9 :
Model compiled
10 :
Model compiled
11 :
Model compiled
12 :
Model compiled
13 :
Model compiled
14 :
Model compiled
15 :
Model compiled
16 :
Model compiled
17 :
Model compiled
18 :
Model compiled
19 :
Model compiled


In [81]:
n=0
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

0
Epoch 1/20
38/38 [==============================] - 13s 269ms/step - loss: 2.1582 - accuracy: 0.2582 - val_loss: 2.9597 - val_accuracy: 0.0998
Epoch 2/20
38/38 [==============================] - 10s 263ms/step - loss: 2.0018 - accuracy: 0.3053 - val_loss: 2.4551 - val_accuracy: 0.0998
Epoch 3/20
38/38 [==============================] - 11s 305ms/step - loss: 1.9678 - accuracy: 0.3060 - val_loss: 2.5033 - val_accuracy: 0.0998
Epoch 4/20
38/38 [==============================] - 12s 324ms/step - loss: 1.9547 - accuracy: 0.3081 - val_loss: 2.5128 - val_accuracy: 0.0998
Epoch 5/20
38/38 [==============================] - 12s 323ms/step - loss: 1.9182 - accuracy: 0.3139 - val_loss: 2.3140 - val_accuracy: 0.0998
Epoch 6/20
38/38 [==============================] - 11s 302ms/step - loss: 1.9261 - accuracy: 0.3004 - val_loss: 2.5121 - val_accuracy: 0.0998
Epoch 7/20
38/38 [==============================] - 11s 299ms/step - loss: 1.9017 - accuracy: 0.3219 - val_loss: 2.3298 - val_accuracy: 0.10

In [82]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

1
Epoch 1/20
75/75 [==============================] - 20s 222ms/step - loss: 2.0682 - accuracy: 0.2583 - val_loss: 2.9472 - val_accuracy: 0.0998
Epoch 2/20
75/75 [==============================] - 16s 216ms/step - loss: 1.9568 - accuracy: 0.3040 - val_loss: 2.4150 - val_accuracy: 0.0998
Epoch 3/20
75/75 [==============================] - 16s 216ms/step - loss: 1.9454 - accuracy: 0.3009 - val_loss: 2.4548 - val_accuracy: 0.0998
Epoch 4/20
75/75 [==============================] - 16s 215ms/step - loss: 1.9434 - accuracy: 0.2941 - val_loss: 2.4094 - val_accuracy: 0.1025
Epoch 5/20
75/75 [==============================] - 16s 215ms/step - loss: 1.9108 - accuracy: 0.2980 - val_loss: 2.2785 - val_accuracy: 0.1329
Epoch 6/20
75/75 [==============================] - 16s 215ms/step - loss: 1.8593 - accuracy: 0.3183 - val_loss: 2.2548 - val_accuracy: 0.1447
Epoch 7/20
75/75 [==============================] - 16s 215ms/step - loss: 1.8511 - accuracy: 0.3228 - val_loss: 2.2029 - val_accuracy: 0.17

In [83]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

2
Epoch 1/20
113/113 [==============================] - 25s 191ms/step - loss: 2.2209 - accuracy: 0.2749 - val_loss: 2.7395 - val_accuracy: 0.0998
Epoch 2/20
113/113 [==============================] - 21s 190ms/step - loss: 1.9821 - accuracy: 0.2953 - val_loss: 2.4671 - val_accuracy: 0.1001
Epoch 3/20
113/113 [==============================] - 21s 189ms/step - loss: 1.9558 - accuracy: 0.2861 - val_loss: 2.4400 - val_accuracy: 0.1074
Epoch 4/20
113/113 [==============================] - 21s 188ms/step - loss: 1.9051 - accuracy: 0.3021 - val_loss: 2.2065 - val_accuracy: 0.1171
Epoch 5/20
113/113 [==============================] - 21s 189ms/step - loss: 1.8719 - accuracy: 0.3104 - val_loss: 2.3991 - val_accuracy: 0.1219
Epoch 6/20
113/113 [==============================] - 21s 190ms/step - loss: 1.8572 - accuracy: 0.3066 - val_loss: 2.4826 - val_accuracy: 0.1247
Epoch 7/20
113/113 [==============================] - 21s 190ms/step - loss: 1.8392 - accuracy: 0.3247 - val_loss: 2.1959 - val_

In [84]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

3
Epoch 1/20
150/150 [==============================] - 30s 177ms/step - loss: 2.1109 - accuracy: 0.2571 - val_loss: 2.4701 - val_accuracy: 0.0998
Epoch 2/20
150/150 [==============================] - 26s 174ms/step - loss: 1.9930 - accuracy: 0.2723 - val_loss: 2.2476 - val_accuracy: 0.1440
Epoch 3/20
150/150 [==============================] - 29s 192ms/step - loss: 1.9295 - accuracy: 0.2851 - val_loss: 2.4340 - val_accuracy: 0.1176
Epoch 4/20
150/150 [==============================] - 27s 183ms/step - loss: 1.9116 - accuracy: 0.2864 - val_loss: 2.1280 - val_accuracy: 0.1737
Epoch 5/20
150/150 [==============================] - 26s 175ms/step - loss: 1.8640 - accuracy: 0.3091 - val_loss: 2.0425 - val_accuracy: 0.2292
Epoch 6/20
150/150 [==============================] - 26s 175ms/step - loss: 1.8389 - accuracy: 0.3180 - val_loss: 1.9783 - val_accuracy: 0.2644
Epoch 7/20
150/150 [==============================] - 26s 174ms/step - loss: 1.8014 - accuracy: 0.3241 - val_loss: 2.1340 - val_

In [85]:
print(n)#
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

4
Epoch 1/20
188/188 [==============================] - 35s 169ms/step - loss: 2.1093 - accuracy: 0.2437 - val_loss: 3.0839 - val_accuracy: 0.1067
Epoch 2/20
188/188 [==============================] - 33s 174ms/step - loss: 2.0014 - accuracy: 0.2564 - val_loss: 2.4630 - val_accuracy: 0.1037
Epoch 3/20
188/188 [==============================] - 32s 169ms/step - loss: 1.9187 - accuracy: 0.2766 - val_loss: 2.1586 - val_accuracy: 0.1771
Epoch 4/20
188/188 [==============================] - 32s 168ms/step - loss: 1.8779 - accuracy: 0.2990 - val_loss: 2.0020 - val_accuracy: 0.2573
Epoch 5/20
188/188 [==============================] - 32s 170ms/step - loss: 1.8090 - accuracy: 0.3153 - val_loss: 1.7603 - val_accuracy: 0.3451
Epoch 6/20
188/188 [==============================] - 32s 169ms/step - loss: 1.7720 - accuracy: 0.3385 - val_loss: 1.7680 - val_accuracy: 0.3659
Epoch 7/20
188/188 [==============================] - 32s 169ms/step - loss: 1.7327 - accuracy: 0.3539 - val_loss: 1.8213 - val_

In [86]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

5
Epoch 1/20
225/225 [==============================] - 40s 164ms/step - loss: 2.1416 - accuracy: 0.2221 - val_loss: 2.1945 - val_accuracy: 0.1436
Epoch 2/20
225/225 [==============================] - 36s 161ms/step - loss: 1.9825 - accuracy: 0.2548 - val_loss: 2.0449 - val_accuracy: 0.2055
Epoch 3/20
225/225 [==============================] - 36s 161ms/step - loss: 1.9158 - accuracy: 0.2827 - val_loss: 1.8675 - val_accuracy: 0.2895
Epoch 4/20
225/225 [==============================] - 36s 161ms/step - loss: 1.8667 - accuracy: 0.3009 - val_loss: 1.9211 - val_accuracy: 0.2794
Epoch 5/20
225/225 [==============================] - 36s 161ms/step - loss: 1.8121 - accuracy: 0.3214 - val_loss: 1.6448 - val_accuracy: 0.4118
Epoch 6/20
225/225 [==============================] - 36s 161ms/step - loss: 1.7579 - accuracy: 0.3453 - val_loss: 1.7237 - val_accuracy: 0.3783
Epoch 7/20
225/225 [==============================] - 36s 160ms/step - loss: 1.7169 - accuracy: 0.3628 - val_loss: 1.7187 - val_

In [87]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

6
Epoch 1/20
263/263 [==============================] - 50s 174ms/step - loss: 2.1564 - accuracy: 0.2034 - val_loss: 2.0607 - val_accuracy: 0.2051
Epoch 2/20
263/263 [==============================] - 46s 174ms/step - loss: 1.9695 - accuracy: 0.2553 - val_loss: 1.9045 - val_accuracy: 0.2856
Epoch 3/20
263/263 [==============================] - 46s 174ms/step - loss: 1.9016 - accuracy: 0.2826 - val_loss: 1.7647 - val_accuracy: 0.3418
Epoch 4/20
263/263 [==============================] - 46s 174ms/step - loss: 1.8343 - accuracy: 0.3083 - val_loss: 1.8688 - val_accuracy: 0.2937
Epoch 5/20
263/263 [==============================] - 46s 174ms/step - loss: 1.7676 - accuracy: 0.3414 - val_loss: 1.8209 - val_accuracy: 0.3455
Epoch 6/20
263/263 [==============================] - 46s 174ms/step - loss: 1.7227 - accuracy: 0.3532 - val_loss: 1.5612 - val_accuracy: 0.4531
Epoch 7/20
263/263 [==============================] - 46s 174ms/step - loss: 1.6749 - accuracy: 0.3680 - val_loss: 1.4951 - val_

In [88]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

7
Epoch 1/20
300/300 [==============================] - 56s 171ms/step - loss: 2.1630 - accuracy: 0.1976 - val_loss: 1.9687 - val_accuracy: 0.2530
Epoch 2/20
300/300 [==============================] - 51s 169ms/step - loss: 1.9627 - accuracy: 0.2542 - val_loss: 1.8449 - val_accuracy: 0.3039
Epoch 3/20
300/300 [==============================] - 51s 169ms/step - loss: 1.8758 - accuracy: 0.2911 - val_loss: 1.6751 - val_accuracy: 0.3700
Epoch 4/20
300/300 [==============================] - 51s 169ms/step - loss: 1.8069 - accuracy: 0.3219 - val_loss: 1.5796 - val_accuracy: 0.4357
Epoch 5/20
300/300 [==============================] - 51s 169ms/step - loss: 1.7390 - accuracy: 0.3441 - val_loss: 1.5554 - val_accuracy: 0.4421
Epoch 6/20
300/300 [==============================] - 51s 170ms/step - loss: 1.6747 - accuracy: 0.3741 - val_loss: 1.5737 - val_accuracy: 0.4022
Epoch 7/20
300/300 [==============================] - 52s 172ms/step - loss: 1.6232 - accuracy: 0.3933 - val_loss: 1.5075 - val_

In [21]:
n=8
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

8
Epoch 1/20
338/338 [==============================] - 75s 132ms/step - loss: 2.1755 - accuracy: 0.1860 - val_loss: 2.0345 - val_accuracy: 0.2359
Epoch 2/20
338/338 [==============================] - 44s 130ms/step - loss: 1.9472 - accuracy: 0.2635 - val_loss: 1.7070 - val_accuracy: 0.3647
Epoch 3/20
338/338 [==============================] - 44s 130ms/step - loss: 1.8415 - accuracy: 0.3063 - val_loss: 1.6621 - val_accuracy: 0.3607
Epoch 4/20
338/338 [==============================] - 44s 130ms/step - loss: 1.7564 - accuracy: 0.3461 - val_loss: 1.6411 - val_accuracy: 0.4153
Epoch 5/20
338/338 [==============================] - 44s 130ms/step - loss: 1.6886 - accuracy: 0.3637 - val_loss: 1.4093 - val_accuracy: 0.5067
Epoch 6/20
338/338 [==============================] - 45s 132ms/step - loss: 1.6194 - accuracy: 0.3999 - val_loss: 1.4261 - val_accuracy: 0.4976
Epoch 7/20
338/338 [==============================] - 46s 136ms/step - loss: 1.5677 - accuracy: 0.4228 - val_loss: 1.3678 - val_

In [22]:
print(n)#
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

9
Epoch 1/20
375/375 [==============================] - 57s 144ms/step - loss: 2.1639 - accuracy: 0.1828 - val_loss: 1.8144 - val_accuracy: 0.3280
Epoch 2/20
375/375 [==============================] - 54s 143ms/step - loss: 1.9290 - accuracy: 0.2728 - val_loss: 1.6767 - val_accuracy: 0.3862
Epoch 3/20
375/375 [==============================] - 53s 142ms/step - loss: 1.8137 - accuracy: 0.3183 - val_loss: 1.6843 - val_accuracy: 0.3691
Epoch 4/20
375/375 [==============================] - 53s 143ms/step - loss: 1.7162 - accuracy: 0.3579 - val_loss: 1.4283 - val_accuracy: 0.4851
Epoch 5/20
375/375 [==============================] - 53s 142ms/step - loss: 1.6327 - accuracy: 0.3890 - val_loss: 1.3447 - val_accuracy: 0.5311
Epoch 6/20
375/375 [==============================] - 54s 143ms/step - loss: 1.5662 - accuracy: 0.4137 - val_loss: 1.3600 - val_accuracy: 0.5244
Epoch 7/20
375/375 [==============================] - 54s 143ms/step - loss: 1.4960 - accuracy: 0.4528 - val_loss: 1.3458 - val_

In [23]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

10
Epoch 1/20
413/413 [==============================] - 68s 153ms/step - loss: 2.1582 - accuracy: 0.1791 - val_loss: 1.8600 - val_accuracy: 0.2988
Epoch 2/20
413/413 [==============================] - 64s 155ms/step - loss: 1.8989 - accuracy: 0.2898 - val_loss: 1.7238 - val_accuracy: 0.3784
Epoch 3/20
413/413 [==============================] - 63s 154ms/step - loss: 1.7827 - accuracy: 0.3365 - val_loss: 1.5089 - val_accuracy: 0.4565
Epoch 4/20
413/413 [==============================] - 63s 153ms/step - loss: 1.6759 - accuracy: 0.3681 - val_loss: 1.5241 - val_accuracy: 0.4479
Epoch 5/20
413/413 [==============================] - 64s 155ms/step - loss: 1.6015 - accuracy: 0.4096 - val_loss: 1.2667 - val_accuracy: 0.5559
Epoch 6/20
413/413 [==============================] - 65s 159ms/step - loss: 1.5095 - accuracy: 0.4414 - val_loss: 1.2384 - val_accuracy: 0.5458
Epoch 7/20
413/413 [==============================] - 64s 154ms/step - loss: 1.4691 - accuracy: 0.4567 - val_loss: 1.2185 - val

In [24]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

11
Epoch 1/20
450/450 [==============================] - 73s 154ms/step - loss: 2.1601 - accuracy: 0.1795 - val_loss: 1.7929 - val_accuracy: 0.3302
Epoch 2/20
450/450 [==============================] - 69s 153ms/step - loss: 1.8602 - accuracy: 0.3044 - val_loss: 1.7839 - val_accuracy: 0.3594
Epoch 3/20
450/450 [==============================] - 68s 151ms/step - loss: 1.7329 - accuracy: 0.3591 - val_loss: 1.4628 - val_accuracy: 0.4942
Epoch 4/20
450/450 [==============================] - 68s 151ms/step - loss: 1.6270 - accuracy: 0.3999 - val_loss: 1.3865 - val_accuracy: 0.5190
Epoch 5/20
450/450 [==============================] - 68s 152ms/step - loss: 1.5341 - accuracy: 0.4309 - val_loss: 1.2302 - val_accuracy: 0.5754
Epoch 6/20
450/450 [==============================] - 68s 152ms/step - loss: 1.4618 - accuracy: 0.4630 - val_loss: 1.1889 - val_accuracy: 0.5859
Epoch 7/20
450/450 [==============================] - 68s 152ms/step - loss: 1.3899 - accuracy: 0.4879 - val_loss: 1.1437 - val

In [25]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

12
Epoch 1/20
488/488 [==============================] - 74s 146ms/step - loss: 2.1566 - accuracy: 0.1810 - val_loss: 1.7411 - val_accuracy: 0.3593
Epoch 2/20
488/488 [==============================] - 68s 140ms/step - loss: 1.8285 - accuracy: 0.3215 - val_loss: 1.5450 - val_accuracy: 0.4402
Epoch 3/20
488/488 [==============================] - 69s 141ms/step - loss: 1.6698 - accuracy: 0.3808 - val_loss: 1.4032 - val_accuracy: 0.4952
Epoch 4/20
488/488 [==============================] - 68s 139ms/step - loss: 1.5437 - accuracy: 0.4314 - val_loss: 1.3236 - val_accuracy: 0.5136
Epoch 5/20
488/488 [==============================] - 68s 139ms/step - loss: 1.4595 - accuracy: 0.4660 - val_loss: 1.2061 - val_accuracy: 0.5868
Epoch 6/20
488/488 [==============================] - 68s 138ms/step - loss: 1.3704 - accuracy: 0.5006 - val_loss: 1.1579 - val_accuracy: 0.5911
Epoch 7/20
488/488 [==============================] - 68s 139ms/step - loss: 1.3009 - accuracy: 0.5302 - val_loss: 1.0607 - val

In [26]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

13
Epoch 1/20
525/525 [==============================] - 83s 150ms/step - loss: 2.1192 - accuracy: 0.1972 - val_loss: 1.6326 - val_accuracy: 0.4172
Epoch 2/20
525/525 [==============================] - 78s 149ms/step - loss: 1.7586 - accuracy: 0.3508 - val_loss: 1.4343 - val_accuracy: 0.4842
Epoch 3/20
525/525 [==============================] - 78s 149ms/step - loss: 1.6047 - accuracy: 0.4133 - val_loss: 1.2955 - val_accuracy: 0.5622
Epoch 4/20
525/525 [==============================] - 79s 151ms/step - loss: 1.4954 - accuracy: 0.4530 - val_loss: 1.1886 - val_accuracy: 0.5648
Epoch 5/20
525/525 [==============================] - 79s 150ms/step - loss: 1.3966 - accuracy: 0.4910 - val_loss: 1.1215 - val_accuracy: 0.6133
Epoch 6/20
525/525 [==============================] - 79s 150ms/step - loss: 1.3197 - accuracy: 0.5262 - val_loss: 1.0324 - val_accuracy: 0.6403
Epoch 7/20
525/525 [==============================] - 78s 149ms/step - loss: 1.2523 - accuracy: 0.5515 - val_loss: 0.9671 - val

In [27]:
print(n)#
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

14
Epoch 1/20
563/563 [==============================] - 88s 149ms/step - loss: 2.1285 - accuracy: 0.1948 - val_loss: 1.6388 - val_accuracy: 0.3986
Epoch 2/20
563/563 [==============================] - 85s 150ms/step - loss: 1.7509 - accuracy: 0.3516 - val_loss: 1.5304 - val_accuracy: 0.4462
Epoch 3/20
563/563 [==============================] - 84s 150ms/step - loss: 1.5596 - accuracy: 0.4309 - val_loss: 1.2937 - val_accuracy: 0.5432
Epoch 4/20
563/563 [==============================] - 84s 150ms/step - loss: 1.4448 - accuracy: 0.4749 - val_loss: 1.1550 - val_accuracy: 0.5959
Epoch 5/20
563/563 [==============================] - 84s 150ms/step - loss: 1.3400 - accuracy: 0.5175 - val_loss: 1.1586 - val_accuracy: 0.6049
Epoch 6/20
563/563 [==============================] - 85s 150ms/step - loss: 1.2603 - accuracy: 0.5502 - val_loss: 1.0481 - val_accuracy: 0.6276
Epoch 7/20
563/563 [==============================] - 84s 150ms/step - loss: 1.2122 - accuracy: 0.5660 - val_loss: 1.0388 - val

In [28]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

15
Epoch 1/20
600/600 [==============================] - 93s 149ms/step - loss: 2.1071 - accuracy: 0.2051 - val_loss: 1.7271 - val_accuracy: 0.3613
Epoch 2/20
600/600 [==============================] - 89s 148ms/step - loss: 1.6849 - accuracy: 0.3789 - val_loss: 1.3184 - val_accuracy: 0.5335
Epoch 3/20
600/600 [==============================] - 89s 148ms/step - loss: 1.4700 - accuracy: 0.4648 - val_loss: 1.1697 - val_accuracy: 0.5882
Epoch 4/20
600/600 [==============================] - 85s 141ms/step - loss: 1.3529 - accuracy: 0.5104 - val_loss: 1.0425 - val_accuracy: 0.6343
Epoch 5/20
600/600 [==============================] - 81s 136ms/step - loss: 1.2501 - accuracy: 0.5565 - val_loss: 0.9955 - val_accuracy: 0.6481
Epoch 6/20
600/600 [==============================] - 82s 137ms/step - loss: 1.1774 - accuracy: 0.5803 - val_loss: 0.9251 - val_accuracy: 0.6944
Epoch 7/20
600/600 [==============================] - 82s 136ms/step - loss: 1.1233 - accuracy: 0.6005 - val_loss: 0.9265 - val

In [29]:
time.sleep(400)

In [30]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

16
Epoch 1/20
638/638 [==============================] - 78s 118ms/step - loss: 2.1018 - accuracy: 0.2124 - val_loss: 1.5245 - val_accuracy: 0.4552
Epoch 2/20
638/638 [==============================] - 75s 118ms/step - loss: 1.6015 - accuracy: 0.4160 - val_loss: 1.2875 - val_accuracy: 0.5387
Epoch 3/20
638/638 [==============================] - 75s 118ms/step - loss: 1.3986 - accuracy: 0.4986 - val_loss: 1.1655 - val_accuracy: 0.5996
Epoch 4/20
638/638 [==============================] - 75s 118ms/step - loss: 1.2837 - accuracy: 0.5450 - val_loss: 1.0071 - val_accuracy: 0.6438
Epoch 5/20
638/638 [==============================] - 76s 118ms/step - loss: 1.1742 - accuracy: 0.5858 - val_loss: 0.9393 - val_accuracy: 0.6809
Epoch 6/20
638/638 [==============================] - 76s 119ms/step - loss: 1.1290 - accuracy: 0.6045 - val_loss: 0.9091 - val_accuracy: 0.6923
Epoch 7/20
638/638 [==============================] - 77s 121ms/step - loss: 1.0600 - accuracy: 0.6265 - val_loss: 0.8652 - val

In [31]:
time.sleep(400)

In [32]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

17
Epoch 1/20
675/675 [==============================] - 94s 133ms/step - loss: 2.0699 - accuracy: 0.2245 - val_loss: 1.5280 - val_accuracy: 0.4544
Epoch 2/20
675/675 [==============================] - 88s 130ms/step - loss: 1.5930 - accuracy: 0.4223 - val_loss: 1.3077 - val_accuracy: 0.5396
Epoch 3/20
675/675 [==============================] - 88s 130ms/step - loss: 1.3888 - accuracy: 0.4978 - val_loss: 1.1179 - val_accuracy: 0.6052
Epoch 4/20
675/675 [==============================] - 90s 133ms/step - loss: 1.2614 - accuracy: 0.5521 - val_loss: 1.0480 - val_accuracy: 0.6352
Epoch 5/20
675/675 [==============================] - 91s 135ms/step - loss: 1.1734 - accuracy: 0.5883 - val_loss: 0.9518 - val_accuracy: 0.6708
Epoch 6/20
675/675 [==============================] - 92s 136ms/step - loss: 1.0880 - accuracy: 0.6183 - val_loss: 0.8956 - val_accuracy: 0.6965
Epoch 7/20
675/675 [==============================] - 96s 143ms/step - loss: 1.0358 - accuracy: 0.6344 - val_loss: 0.8595 - val

In [33]:
time.sleep(400)

In [34]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

18
Epoch 1/20
713/713 [==============================] - 87s 117ms/step - loss: 2.0564 - accuracy: 0.2310 - val_loss: 1.7781 - val_accuracy: 0.3722
Epoch 2/20
713/713 [==============================] - 83s 117ms/step - loss: 1.5244 - accuracy: 0.4527 - val_loss: 1.2283 - val_accuracy: 0.5574
Epoch 3/20
713/713 [==============================] - 84s 118ms/step - loss: 1.3001 - accuracy: 0.5428 - val_loss: 1.2111 - val_accuracy: 0.5826
Epoch 4/20
713/713 [==============================] - 85s 120ms/step - loss: 1.1570 - accuracy: 0.5883 - val_loss: 0.9890 - val_accuracy: 0.6622
Epoch 5/20
713/713 [==============================] - 87s 122ms/step - loss: 1.0768 - accuracy: 0.6208 - val_loss: 1.0189 - val_accuracy: 0.6582
Epoch 6/20
713/713 [==============================] - 88s 123ms/step - loss: 1.0022 - accuracy: 0.6493 - val_loss: 0.8714 - val_accuracy: 0.7031
Epoch 7/20
713/713 [==============================] - 90s 127ms/step - loss: 0.9521 - accuracy: 0.6668 - val_loss: 0.8601 - val

In [35]:
time.sleep(400)

In [36]:
print(n)#
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

19
Epoch 1/20
750/750 [==============================] - 92s 118ms/step - loss: 2.0147 - accuracy: 0.2534 - val_loss: 1.4203 - val_accuracy: 0.4979
Epoch 2/20
750/750 [==============================] - 88s 118ms/step - loss: 1.4501 - accuracy: 0.4796 - val_loss: 1.1929 - val_accuracy: 0.5911
Epoch 3/20
750/750 [==============================] - 89s 118ms/step - loss: 1.2411 - accuracy: 0.5659 - val_loss: 1.0094 - val_accuracy: 0.6477
Epoch 4/20
750/750 [==============================] - 90s 120ms/step - loss: 1.1049 - accuracy: 0.6170 - val_loss: 0.9138 - val_accuracy: 0.6888
Epoch 5/20
750/750 [==============================] - 92s 122ms/step - loss: 1.0129 - accuracy: 0.6478 - val_loss: 0.8635 - val_accuracy: 0.7095
Epoch 6/20
750/750 [==============================] - 94s 125ms/step - loss: 0.9530 - accuracy: 0.6671 - val_loss: 0.8390 - val_accuracy: 0.7167
Epoch 7/20
750/750 [==============================] - 97s 129ms/step - loss: 0.8972 - accuracy: 0.6885 - val_loss: 0.8558 - val

In [37]:
new_model_dg_dir  = "D:/models/aug_22/"+dataset+"/C1/"+dataset+"_model_c1_aug_dg_e1"

i=8

for model in models_dg[8:]:
    model.save(new_model_dg_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_dg_e1_8\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_dg_e1_9\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_dg_e1_10\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_dg_e1_11\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_dg_e1_12\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_dg_e1_13\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_dg_e1_14\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_dg_e1_15\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar

In [ ]:
loading = False

models_dg = []

if loading:
    for i in range(5):
        model_dg_dir = "D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_gn_e1_"+str(i)
        print(model_dg_dir)
        model =utils.My_model('gtsrb',True,model_dg_dir)
        model.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        models_dg.append(model)
       
    

In [38]:
try:
    del deep_gini_values
    del top_images_by_dg
    del top_labels_by_dg
    del image_sets_dg
    del label_sets_dg
    del models_dg
except:
    print("Error")

In [39]:
import gc
gc.collect()

58477

### Softmax 

In [40]:
#softmax values
se_direction = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/softmax_values.npy"
se_values = np.load(se_direction)


In [41]:
# Obtaining top n images by LSA values
top_images_by_se  = utils.get_x_of_indexes(list(np.flip(np.argsort(se_values))),x_train_and_adversary)
top_labels_by_se = utils.get_x_of_indexes(list(np.flip(np.argsort(se_values))),y_train_and_adversary)

In [42]:
m = n_data_points

n = 0
image_sets_se = []
label_sets_se = []

# last
#for i in range(0,len(top_images_by_lsa)//m):

for i in range((len(top_images_by_se)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_se)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_se)%m))
        top_images_by_se_n = np.array(top_images_by_se[:n+m+(len(top_images_by_se)%m)])
        top_labels_by_se_n = np.array(top_labels_by_se[:n+m+(len(top_images_by_se)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_se_n = np.array(top_images_by_se[:n+m])
        top_labels_by_se_n = np.array(top_labels_by_se[:n+m])
    image_sets_se.append(top_images_by_se_n)
    label_sets_se.append(top_labels_by_se_n)
    print(len(top_images_by_se_n))
    n += m



0 :
0  ->  2400
2400
1 :
0  ->  4800
4800
2 :
0  ->  7200
7200
3 :
0  ->  9600
9600
4 :
0  ->  12000
12000
5 :
0  ->  14400
14400
6 :
0  ->  16800
16800
7 :
0  ->  19200
19200
8 :
0  ->  21600
21600
9 :
0  ->  24000
24000
10 :
0  ->  26400
26400
11 :
0  ->  28800
28800
12 :
0  ->  31200
31200
13 :
0  ->  33600
33600
14 :
0  ->  36000
36000
15 :
0  ->  38400
38400
16 :
0  ->  40800
40800
17 :
0  ->  43200
43200
18 :
0  ->  45600
45600
19 :
Last
0  ->  50399
47999


In [43]:
models_se = []
for i in range(len(label_sets_se)):
    print(i,":")
    model = utils.My_model(dataset,False)
    model.compile_model()
    models_se.append(model)

0 :
Model compiled
1 :
Model compiled
2 :
Model compiled
3 :
Model compiled
4 :
Model compiled
5 :
Model compiled
6 :
Model compiled
7 :
Model compiled
8 :
Model compiled
9 :
Model compiled
10 :
Model compiled
11 :
Model compiled
12 :
Model compiled
13 :
Model compiled
14 :
Model compiled
15 :
Model compiled
16 :
Model compiled
17 :
Model compiled
18 :
Model compiled
19 :
Model compiled


In [44]:
n=0
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

0
Epoch 1/20
38/38 [==============================] - 16s 307ms/step - loss: 2.1272 - accuracy: 0.2655 - val_loss: 2.4411 - val_accuracy: 0.0998
Epoch 2/20
38/38 [==============================] - 11s 292ms/step - loss: 1.9865 - accuracy: 0.3158 - val_loss: 2.5051 - val_accuracy: 0.0998
Epoch 3/20
38/38 [==============================] - 12s 328ms/step - loss: 1.9909 - accuracy: 0.3161 - val_loss: 2.6534 - val_accuracy: 0.0998
Epoch 4/20
38/38 [==============================] - 12s 329ms/step - loss: 1.9779 - accuracy: 0.3101 - val_loss: 2.6620 - val_accuracy: 0.0998
Epoch 5/20
38/38 [==============================] - 12s 326ms/step - loss: 1.9827 - accuracy: 0.3023 - val_loss: 2.3886 - val_accuracy: 0.0998
Epoch 6/20
38/38 [==============================] - 12s 326ms/step - loss: 1.9703 - accuracy: 0.3072 - val_loss: 2.5046 - val_accuracy: 0.0998
Epoch 7/20
38/38 [==============================] - 12s 326ms/step - loss: 1.9431 - accuracy: 0.3179 - val_loss: 2.7215 - val_accuracy: 0.09

In [45]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

1
Epoch 1/20
75/75 [==============================] - 23s 248ms/step - loss: 2.1735 - accuracy: 0.2643 - val_loss: 2.4200 - val_accuracy: 0.0998
Epoch 2/20
75/75 [==============================] - 18s 243ms/step - loss: 2.0237 - accuracy: 0.3065 - val_loss: 2.6039 - val_accuracy: 0.0998
Epoch 3/20
75/75 [==============================] - 18s 240ms/step - loss: 1.9875 - accuracy: 0.2988 - val_loss: 2.6308 - val_accuracy: 0.0998
Epoch 4/20
75/75 [==============================] - 18s 240ms/step - loss: 1.9678 - accuracy: 0.3016 - val_loss: 2.3928 - val_accuracy: 0.0998
Epoch 5/20
75/75 [==============================] - 19s 254ms/step - loss: 1.9249 - accuracy: 0.2956 - val_loss: 2.2042 - val_accuracy: 0.0999
Epoch 6/20
75/75 [==============================] - 19s 251ms/step - loss: 1.8859 - accuracy: 0.3242 - val_loss: 2.3881 - val_accuracy: 0.1012
Epoch 7/20
75/75 [==============================] - 18s 243ms/step - loss: 1.8765 - accuracy: 0.3232 - val_loss: 2.2590 - val_accuracy: 0.11

In [46]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

2
Epoch 1/20
113/113 [==============================] - 27s 206ms/step - loss: 2.1978 - accuracy: 0.2745 - val_loss: 2.4186 - val_accuracy: 0.0998
Epoch 2/20
113/113 [==============================] - 23s 208ms/step - loss: 1.9881 - accuracy: 0.2939 - val_loss: 2.4880 - val_accuracy: 0.0998
Epoch 3/20
113/113 [==============================] - 23s 207ms/step - loss: 1.9627 - accuracy: 0.2797 - val_loss: 2.4844 - val_accuracy: 0.1002
Epoch 4/20
113/113 [==============================] - 23s 206ms/step - loss: 1.9253 - accuracy: 0.2944 - val_loss: 2.2326 - val_accuracy: 0.1431
Epoch 5/20
113/113 [==============================] - 23s 208ms/step - loss: 1.8800 - accuracy: 0.3095 - val_loss: 2.2350 - val_accuracy: 0.1268
Epoch 6/20
113/113 [==============================] - 21s 190ms/step - loss: 1.8582 - accuracy: 0.3065 - val_loss: 2.6399 - val_accuracy: 0.1077
Epoch 7/20
113/113 [==============================] - 21s 189ms/step - loss: 1.8324 - accuracy: 0.3177 - val_loss: 2.2453 - val_

In [47]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

3
Epoch 1/20
150/150 [==============================] - 30s 180ms/step - loss: 2.0939 - accuracy: 0.2523 - val_loss: 2.3596 - val_accuracy: 0.0998
Epoch 2/20
150/150 [==============================] - 27s 177ms/step - loss: 1.9769 - accuracy: 0.2797 - val_loss: 2.2591 - val_accuracy: 0.1205
Epoch 3/20
150/150 [==============================] - 27s 177ms/step - loss: 1.9405 - accuracy: 0.2840 - val_loss: 2.1502 - val_accuracy: 0.1600
Epoch 4/20
150/150 [==============================] - 26s 176ms/step - loss: 1.9027 - accuracy: 0.2900 - val_loss: 2.0897 - val_accuracy: 0.1903
Epoch 5/20
150/150 [==============================] - 26s 176ms/step - loss: 1.8704 - accuracy: 0.3039 - val_loss: 2.1562 - val_accuracy: 0.1993
Epoch 6/20
150/150 [==============================] - 26s 177ms/step - loss: 1.8382 - accuracy: 0.3242 - val_loss: 1.9661 - val_accuracy: 0.2587
Epoch 7/20
150/150 [==============================] - 28s 184ms/step - loss: 1.7974 - accuracy: 0.3356 - val_loss: 1.8796 - val_

In [48]:
print(n)#
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

4
Epoch 1/20
188/188 [==============================] - 35s 169ms/step - loss: 2.0956 - accuracy: 0.2458 - val_loss: 2.5119 - val_accuracy: 0.1050
Epoch 2/20
188/188 [==============================] - 31s 167ms/step - loss: 1.9810 - accuracy: 0.2605 - val_loss: 2.1734 - val_accuracy: 0.1803
Epoch 3/20
188/188 [==============================] - 31s 167ms/step - loss: 1.9184 - accuracy: 0.2859 - val_loss: 2.1269 - val_accuracy: 0.1846
Epoch 4/20
188/188 [==============================] - 31s 167ms/step - loss: 1.8876 - accuracy: 0.2984 - val_loss: 1.8665 - val_accuracy: 0.3304
Epoch 5/20
188/188 [==============================] - 32s 170ms/step - loss: 1.8437 - accuracy: 0.3034 - val_loss: 1.8163 - val_accuracy: 0.3281
Epoch 6/20
188/188 [==============================] - 31s 167ms/step - loss: 1.7893 - accuracy: 0.3294 - val_loss: 1.7924 - val_accuracy: 0.3423
Epoch 7/20
188/188 [==============================] - 31s 167ms/step - loss: 1.7622 - accuracy: 0.3481 - val_loss: 2.0212 - val_

In [49]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

5
Epoch 1/20
225/225 [==============================] - 43s 179ms/step - loss: 2.1056 - accuracy: 0.2222 - val_loss: 2.1947 - val_accuracy: 0.1104
Epoch 2/20
225/225 [==============================] - 40s 178ms/step - loss: 1.9783 - accuracy: 0.2543 - val_loss: 2.1552 - val_accuracy: 0.1765
Epoch 3/20
225/225 [==============================] - 37s 165ms/step - loss: 1.9138 - accuracy: 0.2726 - val_loss: 1.8247 - val_accuracy: 0.3449
Epoch 4/20
225/225 [==============================] - 37s 163ms/step - loss: 1.8581 - accuracy: 0.3002 - val_loss: 1.7532 - val_accuracy: 0.3657
Epoch 5/20
225/225 [==============================] - 37s 164ms/step - loss: 1.7984 - accuracy: 0.3282 - val_loss: 1.7275 - val_accuracy: 0.3711
Epoch 6/20
225/225 [==============================] - 37s 164ms/step - loss: 1.7382 - accuracy: 0.3515 - val_loss: 1.8603 - val_accuracy: 0.3187
Epoch 7/20
225/225 [==============================] - 37s 164ms/step - loss: 1.7040 - accuracy: 0.3596 - val_loss: 1.6802 - val_

In [50]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

6
Epoch 1/20
263/263 [==============================] - 44s 156ms/step - loss: 2.1412 - accuracy: 0.2162 - val_loss: 2.0163 - val_accuracy: 0.2629
Epoch 2/20
263/263 [==============================] - 42s 159ms/step - loss: 1.9743 - accuracy: 0.2504 - val_loss: 2.0004 - val_accuracy: 0.2370
Epoch 3/20
263/263 [==============================] - 41s 156ms/step - loss: 1.9035 - accuracy: 0.2901 - val_loss: 1.7479 - val_accuracy: 0.3318
Epoch 4/20
263/263 [==============================] - 41s 156ms/step - loss: 1.8230 - accuracy: 0.3179 - val_loss: 1.8992 - val_accuracy: 0.3017
Epoch 5/20
263/263 [==============================] - 41s 156ms/step - loss: 1.7596 - accuracy: 0.3404 - val_loss: 1.7123 - val_accuracy: 0.3567
Epoch 6/20
263/263 [==============================] - 41s 156ms/step - loss: 1.7162 - accuracy: 0.3638 - val_loss: 1.6107 - val_accuracy: 0.3952
Epoch 7/20
263/263 [==============================] - 41s 156ms/step - loss: 1.6520 - accuracy: 0.3877 - val_loss: 1.4785 - val_

In [51]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

7
Epoch 1/20
300/300 [==============================] - 57s 176ms/step - loss: 2.1474 - accuracy: 0.2046 - val_loss: 2.0271 - val_accuracy: 0.2423
Epoch 2/20
300/300 [==============================] - 50s 167ms/step - loss: 1.9667 - accuracy: 0.2537 - val_loss: 1.8921 - val_accuracy: 0.2945
Epoch 3/20
300/300 [==============================] - 50s 168ms/step - loss: 1.8760 - accuracy: 0.2898 - val_loss: 1.8673 - val_accuracy: 0.3063
Epoch 4/20
300/300 [==============================] - 50s 168ms/step - loss: 1.8036 - accuracy: 0.3219 - val_loss: 1.6507 - val_accuracy: 0.4281
Epoch 5/20
300/300 [==============================] - 50s 168ms/step - loss: 1.7335 - accuracy: 0.3579 - val_loss: 1.5753 - val_accuracy: 0.4333
Epoch 6/20
300/300 [==============================] - 50s 167ms/step - loss: 1.6618 - accuracy: 0.3793 - val_loss: 1.4675 - val_accuracy: 0.4787
Epoch 7/20
300/300 [==============================] - 50s 168ms/step - loss: 1.5902 - accuracy: 0.4077 - val_loss: 1.4864 - val_

In [52]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

8
Epoch 1/20
338/338 [==============================] - 54s 152ms/step - loss: 2.1504 - accuracy: 0.1959 - val_loss: 1.8984 - val_accuracy: 0.2627
Epoch 2/20
338/338 [==============================] - 52s 153ms/step - loss: 1.9389 - accuracy: 0.2594 - val_loss: 1.7016 - val_accuracy: 0.3607
Epoch 3/20
338/338 [==============================] - 51s 152ms/step - loss: 1.8468 - accuracy: 0.3058 - val_loss: 1.6690 - val_accuracy: 0.3567
Epoch 4/20
338/338 [==============================] - 52s 153ms/step - loss: 1.7364 - accuracy: 0.3442 - val_loss: 1.5082 - val_accuracy: 0.4378
Epoch 5/20
338/338 [==============================] - 51s 150ms/step - loss: 1.6725 - accuracy: 0.3775 - val_loss: 1.4309 - val_accuracy: 0.4787
Epoch 6/20
338/338 [==============================] - 51s 151ms/step - loss: 1.5983 - accuracy: 0.3988 - val_loss: 1.5184 - val_accuracy: 0.4434
Epoch 7/20
338/338 [==============================] - 51s 151ms/step - loss: 1.5414 - accuracy: 0.4363 - val_loss: 1.3127 - val_

In [53]:
print(n)#
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

9
Epoch 1/20
375/375 [==============================] - 62s 156ms/step - loss: 2.1644 - accuracy: 0.1833 - val_loss: 1.9447 - val_accuracy: 0.2700
Epoch 2/20
375/375 [==============================] - 58s 154ms/step - loss: 1.9236 - accuracy: 0.2748 - val_loss: 1.6794 - val_accuracy: 0.4135
Epoch 3/20
375/375 [==============================] - 58s 153ms/step - loss: 1.8026 - accuracy: 0.3235 - val_loss: 1.6212 - val_accuracy: 0.4201
Epoch 4/20
375/375 [==============================] - 60s 159ms/step - loss: 1.7170 - accuracy: 0.3637 - val_loss: 1.4565 - val_accuracy: 0.4687
Epoch 5/20
375/375 [==============================] - 63s 169ms/step - loss: 1.6341 - accuracy: 0.3952 - val_loss: 1.4407 - val_accuracy: 0.4877
Epoch 6/20
375/375 [==============================] - 64s 170ms/step - loss: 1.5678 - accuracy: 0.4220 - val_loss: 1.3095 - val_accuracy: 0.5422
Epoch 7/20
375/375 [==============================] - 63s 169ms/step - loss: 1.4927 - accuracy: 0.4447 - val_loss: 1.2139 - val_

In [54]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

10
Epoch 1/20
413/413 [==============================] - 75s 171ms/step - loss: 2.1472 - accuracy: 0.1859 - val_loss: 1.8896 - val_accuracy: 0.2962
Epoch 2/20
413/413 [==============================] - 70s 171ms/step - loss: 1.8842 - accuracy: 0.2956 - val_loss: 1.5862 - val_accuracy: 0.4174
Epoch 3/20
413/413 [==============================] - 71s 171ms/step - loss: 1.7589 - accuracy: 0.3414 - val_loss: 1.4863 - val_accuracy: 0.4659
Epoch 4/20
413/413 [==============================] - 70s 170ms/step - loss: 1.6690 - accuracy: 0.3794 - val_loss: 1.4781 - val_accuracy: 0.4322
Epoch 5/20
413/413 [==============================] - 70s 170ms/step - loss: 1.5848 - accuracy: 0.4116 - val_loss: 1.2751 - val_accuracy: 0.5402
Epoch 6/20
413/413 [==============================] - 70s 170ms/step - loss: 1.4925 - accuracy: 0.4465 - val_loss: 1.2624 - val_accuracy: 0.5419
Epoch 7/20
413/413 [==============================] - 72s 174ms/step - loss: 1.4368 - accuracy: 0.4735 - val_loss: 1.1755 - val

In [55]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

11
Epoch 1/20
450/450 [==============================] - 79s 165ms/step - loss: 2.1595 - accuracy: 0.1866 - val_loss: 1.7389 - val_accuracy: 0.3842
Epoch 2/20
450/450 [==============================] - 68s 152ms/step - loss: 1.8579 - accuracy: 0.3055 - val_loss: 1.6222 - val_accuracy: 0.4162
Epoch 3/20
450/450 [==============================] - 68s 152ms/step - loss: 1.7223 - accuracy: 0.3576 - val_loss: 1.5060 - val_accuracy: 0.4735
Epoch 4/20
450/450 [==============================] - 69s 154ms/step - loss: 1.6170 - accuracy: 0.3990 - val_loss: 1.2717 - val_accuracy: 0.5541
Epoch 5/20
450/450 [==============================] - 69s 152ms/step - loss: 1.5136 - accuracy: 0.4420 - val_loss: 1.2390 - val_accuracy: 0.5569
Epoch 6/20
450/450 [==============================] - 68s 152ms/step - loss: 1.4482 - accuracy: 0.4668 - val_loss: 1.1344 - val_accuracy: 0.6074
Epoch 7/20
450/450 [==============================] - 68s 151ms/step - loss: 1.3885 - accuracy: 0.4921 - val_loss: 1.0772 - val

In [56]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

12
Epoch 1/20
488/488 [==============================] - 77s 151ms/step - loss: 2.1498 - accuracy: 0.1869 - val_loss: 1.6762 - val_accuracy: 0.3820
Epoch 2/20
488/488 [==============================] - 74s 152ms/step - loss: 1.8149 - accuracy: 0.3238 - val_loss: 1.5242 - val_accuracy: 0.4591
Epoch 3/20
488/488 [==============================] - 79s 162ms/step - loss: 1.6707 - accuracy: 0.3861 - val_loss: 1.3168 - val_accuracy: 0.5227
Epoch 4/20
488/488 [==============================] - 82s 167ms/step - loss: 1.5609 - accuracy: 0.4312 - val_loss: 1.3616 - val_accuracy: 0.4997
Epoch 5/20
488/488 [==============================] - 78s 161ms/step - loss: 1.4703 - accuracy: 0.4611 - val_loss: 1.1677 - val_accuracy: 0.5972
Epoch 6/20
488/488 [==============================] - 75s 153ms/step - loss: 1.3815 - accuracy: 0.4962 - val_loss: 1.1395 - val_accuracy: 0.5943
Epoch 7/20
488/488 [==============================] - 74s 152ms/step - loss: 1.3197 - accuracy: 0.5220 - val_loss: 1.1017 - val

In [58]:
n=13
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

13
Epoch 1/20
525/525 [==============================] - 74s 136ms/step - loss: 2.1328 - accuracy: 0.1884 - val_loss: 1.6108 - val_accuracy: 0.4276
Epoch 2/20
525/525 [==============================] - 70s 133ms/step - loss: 1.7621 - accuracy: 0.3468 - val_loss: 1.4658 - val_accuracy: 0.4802
Epoch 3/20
525/525 [==============================] - 70s 134ms/step - loss: 1.5962 - accuracy: 0.4197 - val_loss: 1.2742 - val_accuracy: 0.5586
Epoch 4/20
525/525 [==============================] - 70s 133ms/step - loss: 1.4728 - accuracy: 0.4634 - val_loss: 1.2107 - val_accuracy: 0.5733
Epoch 5/20
525/525 [==============================] - 70s 133ms/step - loss: 1.3793 - accuracy: 0.5015 - val_loss: 1.0780 - val_accuracy: 0.6171
Epoch 6/20
525/525 [==============================] - 71s 135ms/step - loss: 1.3043 - accuracy: 0.5309 - val_loss: 1.0644 - val_accuracy: 0.6263
Epoch 7/20
525/525 [==============================] - 74s 140ms/step - loss: 1.2500 - accuracy: 0.5540 - val_loss: 1.0168 - val

In [59]:
print(n)#
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

14
Epoch 1/20
563/563 [==============================] - 92s 157ms/step - loss: 2.1337 - accuracy: 0.1898 - val_loss: 1.7149 - val_accuracy: 0.3655
Epoch 2/20
563/563 [==============================] - 94s 166ms/step - loss: 1.7731 - accuracy: 0.3433 - val_loss: 1.4953 - val_accuracy: 0.4614
Epoch 3/20
563/563 [==============================] - 94s 166ms/step - loss: 1.5879 - accuracy: 0.4212 - val_loss: 1.2929 - val_accuracy: 0.5441
Epoch 4/20
563/563 [==============================] - 93s 166ms/step - loss: 1.4777 - accuracy: 0.4632 - val_loss: 1.1962 - val_accuracy: 0.5658
Epoch 5/20
563/563 [==============================] - 94s 166ms/step - loss: 1.3831 - accuracy: 0.4983 - val_loss: 1.0863 - val_accuracy: 0.6159
Epoch 6/20
563/563 [==============================] - 94s 167ms/step - loss: 1.2866 - accuracy: 0.5396 - val_loss: 1.0008 - val_accuracy: 0.6431
Epoch 7/20
563/563 [==============================] - 92s 164ms/step - loss: 1.2167 - accuracy: 0.5589 - val_loss: 0.9884 - val

In [60]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

15
Epoch 1/20
600/600 [==============================] - 93s 149ms/step - loss: 2.0978 - accuracy: 0.2136 - val_loss: 1.6595 - val_accuracy: 0.4019
Epoch 2/20
600/600 [==============================] - 89s 148ms/step - loss: 1.6915 - accuracy: 0.3795 - val_loss: 1.3762 - val_accuracy: 0.5113
Epoch 3/20
600/600 [==============================] - 91s 151ms/step - loss: 1.4996 - accuracy: 0.4596 - val_loss: 1.3082 - val_accuracy: 0.5271
Epoch 4/20
600/600 [==============================] - 89s 148ms/step - loss: 1.3888 - accuracy: 0.4991 - val_loss: 1.0734 - val_accuracy: 0.6302
Epoch 5/20
600/600 [==============================] - 96s 160ms/step - loss: 1.2828 - accuracy: 0.5421 - val_loss: 1.0203 - val_accuracy: 0.6449
Epoch 6/20
600/600 [==============================] - 98s 164ms/step - loss: 1.2097 - accuracy: 0.5676 - val_loss: 0.9661 - val_accuracy: 0.6625
Epoch 7/20
600/600 [==============================] - 99s 164ms/step - loss: 1.1503 - accuracy: 0.5931 - val_loss: 1.0083 - val

In [61]:
time.sleep(400)

In [62]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

16
Epoch 1/20
638/638 [==============================] - 87s 131ms/step - loss: 2.0847 - accuracy: 0.2228 - val_loss: 1.6093 - val_accuracy: 0.4027
Epoch 2/20
638/638 [==============================] - 84s 132ms/step - loss: 1.6093 - accuracy: 0.4125 - val_loss: 1.2574 - val_accuracy: 0.5610
Epoch 3/20
638/638 [==============================] - 84s 131ms/step - loss: 1.4132 - accuracy: 0.4918 - val_loss: 1.0793 - val_accuracy: 0.6286
Epoch 4/20
638/638 [==============================] - 84s 131ms/step - loss: 1.2774 - accuracy: 0.5455 - val_loss: 1.0175 - val_accuracy: 0.6433
Epoch 5/20
638/638 [==============================] - 84s 131ms/step - loss: 1.1959 - accuracy: 0.5742 - val_loss: 0.9131 - val_accuracy: 0.6849
Epoch 6/20
638/638 [==============================] - 85s 133ms/step - loss: 1.1107 - accuracy: 0.6066 - val_loss: 0.9644 - val_accuracy: 0.6632
Epoch 7/20
638/638 [==============================] - 86s 135ms/step - loss: 1.0626 - accuracy: 0.6276 - val_loss: 0.8818 - val

In [63]:
time.sleep(400)

In [64]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

17
Epoch 1/20
675/675 [==============================] - 104s 148ms/step - loss: 2.0636 - accuracy: 0.2330 - val_loss: 1.4308 - val_accuracy: 0.4819
Epoch 2/20
675/675 [==============================] - 99s 147ms/step - loss: 1.5253 - accuracy: 0.4464 - val_loss: 1.2755 - val_accuracy: 0.5514
Epoch 3/20
675/675 [==============================] - 99s 147ms/step - loss: 1.3372 - accuracy: 0.5279 - val_loss: 1.0507 - val_accuracy: 0.6405
Epoch 4/20
675/675 [==============================] - 99s 147ms/step - loss: 1.1862 - accuracy: 0.5800 - val_loss: 0.9231 - val_accuracy: 0.6827
Epoch 5/20
675/675 [==============================] - 99s 147ms/step - loss: 1.1151 - accuracy: 0.6101 - val_loss: 0.8950 - val_accuracy: 0.7000
Epoch 6/20
675/675 [==============================] - 101s 149ms/step - loss: 1.0454 - accuracy: 0.6345 - val_loss: 0.8537 - val_accuracy: 0.7078
Epoch 7/20
675/675 [==============================] - 103s 153ms/step - loss: 0.9918 - accuracy: 0.6566 - val_loss: 0.7953 - 

In [65]:
time.sleep(400)

In [66]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

18
Epoch 1/20
713/713 [==============================] - 111s 149ms/step - loss: 2.0403 - accuracy: 0.2385 - val_loss: 1.4758 - val_accuracy: 0.4735
Epoch 2/20
713/713 [==============================] - 106s 148ms/step - loss: 1.5073 - accuracy: 0.4560 - val_loss: 1.3390 - val_accuracy: 0.5305
Epoch 3/20
713/713 [==============================] - 105s 148ms/step - loss: 1.3069 - accuracy: 0.5356 - val_loss: 1.1004 - val_accuracy: 0.6261
Epoch 4/20
713/713 [==============================] - 106s 148ms/step - loss: 1.1720 - accuracy: 0.5884 - val_loss: 0.9651 - val_accuracy: 0.6618
Epoch 5/20
713/713 [==============================] - 106s 148ms/step - loss: 1.0767 - accuracy: 0.6254 - val_loss: 0.9599 - val_accuracy: 0.6699
Epoch 6/20
713/713 [==============================] - 108s 151ms/step - loss: 1.0056 - accuracy: 0.6519 - val_loss: 0.9296 - val_accuracy: 0.6845
Epoch 7/20
713/713 [==============================] - 111s 155ms/step - loss: 0.9669 - accuracy: 0.6646 - val_loss: 1.002

In [67]:
time.sleep(400)

In [68]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

19
Epoch 1/20
750/750 [==============================] - 115s 147ms/step - loss: 2.0276 - accuracy: 0.2497 - val_loss: 1.4472 - val_accuracy: 0.4837
Epoch 2/20
750/750 [==============================] - 110s 147ms/step - loss: 1.4656 - accuracy: 0.4732 - val_loss: 1.2133 - val_accuracy: 0.5763
Epoch 3/20
750/750 [==============================] - 110s 147ms/step - loss: 1.2339 - accuracy: 0.5663 - val_loss: 1.0324 - val_accuracy: 0.6380
Epoch 4/20
750/750 [==============================] - 110s 146ms/step - loss: 1.0925 - accuracy: 0.6158 - val_loss: 0.9177 - val_accuracy: 0.6835
Epoch 5/20
750/750 [==============================] - 110s 147ms/step - loss: 1.0036 - accuracy: 0.6515 - val_loss: 0.8800 - val_accuracy: 0.6962
Epoch 6/20
750/750 [==============================] - 112s 150ms/step - loss: 0.9442 - accuracy: 0.6709 - val_loss: 0.8186 - val_accuracy: 0.7155
Epoch 7/20
750/750 [==============================] - 115s 154ms/step - loss: 0.9016 - accuracy: 0.6875 - val_loss: 0.801

In [ ]:
len(models_se)

In [69]:
new_model_se_dir  = "D:/models/aug_22/"+dataset+"/C1/"+dataset+"_model_c1_aug_se_e1"

i=13

for model in models_se[13:]:
    model.save(new_model_se_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_se_e1_13\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_se_e1_14\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_se_e1_15\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_se_e1_16\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_se_e1_17\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_se_e1_18\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_se_e1_19\assets
Model has been saved


In [70]:
try:
    del se_values
    del top_images_by_se
    del top_labels_by_se
    del image_sets_se
    del label_sets_se
    del models_se
except:
    print("Error")

In [71]:
gc.collect()

95347

## Training guided by Random values

In [ ]:
import random
random_indexes =list(range(len(x_train_and_adversary)))
random.shuffle(random_indexes)
print(random_indexes[:10])
print(len(random_indexes))

In [72]:
save_dir = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/random_values.npy"


random_indexes = np.load(save_dir)

In [73]:
len(random_indexes)

47999

In [74]:
# Obtaining top n images by random values
top_images_by_random = utils.get_x_of_indexes(list(np.flip(np.argsort(random_indexes))),x_train_and_adversary)
top_labels_by_random = utils.get_x_of_indexes(list(np.flip(np.argsort(random_indexes))),y_train_and_adversary)

In [75]:
m = n_data_points
n = 0
image_sets_random = []
label_sets_random = []



for i in range((len(top_images_by_random)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_random)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_random)%m))
        top_images_by_random_n = np.array(top_images_by_random[:n+m+(len(top_images_by_random)%m)])
        top_labels_by_random_n = np.array(top_labels_by_random[:n+m+(len(top_images_by_random)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_random_n = np.array(top_images_by_random[:n+m])
        top_labels_by_random_n = np.array(top_labels_by_random[:n+m])
    image_sets_random.append(top_images_by_random_n)
    label_sets_random.append(top_labels_by_random_n)
    print(len(top_images_by_random_n))
    n += m




0 :
0  ->  2400
2400
1 :
0  ->  4800
4800
2 :
0  ->  7200
7200
3 :
0  ->  9600
9600
4 :
0  ->  12000
12000
5 :
0  ->  14400
14400
6 :
0  ->  16800
16800
7 :
0  ->  19200
19200
8 :
0  ->  21600
21600
9 :
0  ->  24000
24000
10 :
0  ->  26400
26400
11 :
0  ->  28800
28800
12 :
0  ->  31200
31200
13 :
0  ->  33600
33600
14 :
0  ->  36000
36000
15 :
0  ->  38400
38400
16 :
0  ->  40800
40800
17 :
0  ->  43200
43200
18 :
0  ->  45600
45600
19 :
Last
0  ->  50399
47999


In [76]:
len(image_sets_random)

20

In [77]:
print(model_dir)

models_random = []
for i in range(len(label_sets_random)):
    print(i,":")
    model = utils.My_model(dataset,False,None)
    model.compile_model()
    models_random.append(model)


C:/Users/fjdur/Documents/upc-july/models/model_02
0 :
Model compiled
1 :
Model compiled
2 :
Model compiled
3 :
Model compiled
4 :
Model compiled
5 :
Model compiled
6 :
Model compiled
7 :
Model compiled
8 :
Model compiled
9 :
Model compiled
10 :
Model compiled
11 :
Model compiled
12 :
Model compiled
13 :
Model compiled
14 :
Model compiled
15 :
Model compiled
16 :
Model compiled
17 :
Model compiled
18 :
Model compiled
19 :
Model compiled


In [78]:
n=0
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

0
Epoch 1/20
38/38 [==============================] - 13s 267ms/step - loss: 2.3121 - accuracy: 0.1016 - val_loss: 2.2993 - val_accuracy: 0.0998
Epoch 2/20
38/38 [==============================] - 11s 289ms/step - loss: 2.2627 - accuracy: 0.1640 - val_loss: 2.0669 - val_accuracy: 0.2533
Epoch 3/20
38/38 [==============================] - 11s 303ms/step - loss: 2.0810 - accuracy: 0.2335 - val_loss: 2.0162 - val_accuracy: 0.2532
Epoch 4/20
38/38 [==============================] - 11s 301ms/step - loss: 1.9602 - accuracy: 0.2803 - val_loss: 1.9683 - val_accuracy: 0.3006
Epoch 5/20
38/38 [==============================] - 11s 301ms/step - loss: 1.9234 - accuracy: 0.3082 - val_loss: 1.8884 - val_accuracy: 0.3269
Epoch 6/20
38/38 [==============================] - 11s 305ms/step - loss: 1.8560 - accuracy: 0.3167 - val_loss: 1.8719 - val_accuracy: 0.3261
Epoch 7/20
38/38 [==============================] - 11s 300ms/step - loss: 1.7979 - accuracy: 0.3462 - val_loss: 1.7415 - val_accuracy: 0.37

In [79]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

1
Epoch 1/20
75/75 [==============================] - 19s 218ms/step - loss: 2.2965 - accuracy: 0.1185 - val_loss: 2.0683 - val_accuracy: 0.2512
Epoch 2/20
75/75 [==============================] - 16s 211ms/step - loss: 2.0632 - accuracy: 0.2514 - val_loss: 1.8942 - val_accuracy: 0.3144
Epoch 3/20
75/75 [==============================] - 16s 211ms/step - loss: 1.9389 - accuracy: 0.2930 - val_loss: 1.8009 - val_accuracy: 0.3471
Epoch 4/20
75/75 [==============================] - 16s 211ms/step - loss: 1.8125 - accuracy: 0.3318 - val_loss: 1.8144 - val_accuracy: 0.3458
Epoch 5/20
75/75 [==============================] - 16s 213ms/step - loss: 1.7093 - accuracy: 0.3801 - val_loss: 1.6871 - val_accuracy: 0.3954
Epoch 6/20
75/75 [==============================] - 16s 211ms/step - loss: 1.6661 - accuracy: 0.3998 - val_loss: 1.5669 - val_accuracy: 0.4342
Epoch 7/20
75/75 [==============================] - 16s 211ms/step - loss: 1.5750 - accuracy: 0.4356 - val_loss: 1.6658 - val_accuracy: 0.41

In [80]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

2
Epoch 1/20
113/113 [==============================] - 27s 200ms/step - loss: 2.2678 - accuracy: 0.1333 - val_loss: 1.9780 - val_accuracy: 0.2843
Epoch 2/20
113/113 [==============================] - 23s 201ms/step - loss: 1.9644 - accuracy: 0.2772 - val_loss: 1.9392 - val_accuracy: 0.3072
Epoch 3/20
113/113 [==============================] - 23s 200ms/step - loss: 1.8263 - accuracy: 0.3353 - val_loss: 1.8540 - val_accuracy: 0.3317
Epoch 4/20
113/113 [==============================] - 23s 200ms/step - loss: 1.7148 - accuracy: 0.3713 - val_loss: 1.6341 - val_accuracy: 0.3967
Epoch 5/20
113/113 [==============================] - 23s 201ms/step - loss: 1.6244 - accuracy: 0.4108 - val_loss: 1.7739 - val_accuracy: 0.3642
Epoch 6/20
113/113 [==============================] - 23s 200ms/step - loss: 1.5488 - accuracy: 0.4497 - val_loss: 1.4619 - val_accuracy: 0.4690
Epoch 7/20
113/113 [==============================] - 23s 201ms/step - loss: 1.4831 - accuracy: 0.4659 - val_loss: 1.4280 - val_

In [81]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

3
Epoch 1/20
150/150 [==============================] - 33s 190ms/step - loss: 2.2518 - accuracy: 0.1463 - val_loss: 1.9125 - val_accuracy: 0.3134
Epoch 2/20
150/150 [==============================] - 28s 187ms/step - loss: 1.9421 - accuracy: 0.2907 - val_loss: 1.6895 - val_accuracy: 0.3848
Epoch 3/20
150/150 [==============================] - 28s 186ms/step - loss: 1.7589 - accuracy: 0.3661 - val_loss: 1.6892 - val_accuracy: 0.3943
Epoch 4/20
150/150 [==============================] - 28s 186ms/step - loss: 1.6235 - accuracy: 0.4084 - val_loss: 1.4934 - val_accuracy: 0.4618
Epoch 5/20
150/150 [==============================] - 28s 187ms/step - loss: 1.4993 - accuracy: 0.4572 - val_loss: 1.4146 - val_accuracy: 0.5024
Epoch 6/20
150/150 [==============================] - 28s 186ms/step - loss: 1.4054 - accuracy: 0.4951 - val_loss: 1.3593 - val_accuracy: 0.5203
Epoch 7/20
150/150 [==============================] - 28s 189ms/step - loss: 1.3333 - accuracy: 0.5243 - val_loss: 1.2700 - val_

In [82]:
print(n)#
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

4
Epoch 1/20
188/188 [==============================] - 34s 164ms/step - loss: 2.2203 - accuracy: 0.1520 - val_loss: 2.0950 - val_accuracy: 0.2473
Epoch 2/20
188/188 [==============================] - 31s 164ms/step - loss: 1.8794 - accuracy: 0.3065 - val_loss: 1.7620 - val_accuracy: 0.3508
Epoch 3/20
188/188 [==============================] - 31s 164ms/step - loss: 1.7033 - accuracy: 0.3815 - val_loss: 1.6894 - val_accuracy: 0.3741
Epoch 4/20
188/188 [==============================] - 31s 163ms/step - loss: 1.5993 - accuracy: 0.4194 - val_loss: 1.8207 - val_accuracy: 0.3596
Epoch 5/20
188/188 [==============================] - 31s 163ms/step - loss: 1.5044 - accuracy: 0.4609 - val_loss: 1.3969 - val_accuracy: 0.5137
Epoch 6/20
188/188 [==============================] - 31s 165ms/step - loss: 1.3665 - accuracy: 0.5146 - val_loss: 1.2844 - val_accuracy: 0.5403
Epoch 7/20
188/188 [==============================] - 31s 165ms/step - loss: 1.2834 - accuracy: 0.5459 - val_loss: 1.2593 - val_

In [83]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

5
Epoch 1/20
225/225 [==============================] - 39s 160ms/step - loss: 2.1859 - accuracy: 0.1800 - val_loss: 1.8262 - val_accuracy: 0.3407
Epoch 2/20
225/225 [==============================] - 36s 158ms/step - loss: 1.8012 - accuracy: 0.3405 - val_loss: 1.5429 - val_accuracy: 0.4393
Epoch 3/20
225/225 [==============================] - 36s 162ms/step - loss: 1.6091 - accuracy: 0.4125 - val_loss: 1.4085 - val_accuracy: 0.4995
Epoch 4/20
225/225 [==============================] - 36s 158ms/step - loss: 1.4718 - accuracy: 0.4718 - val_loss: 1.3374 - val_accuracy: 0.5222
Epoch 5/20
225/225 [==============================] - 35s 158ms/step - loss: 1.3608 - accuracy: 0.5169 - val_loss: 1.2328 - val_accuracy: 0.5695
Epoch 6/20
225/225 [==============================] - 35s 157ms/step - loss: 1.2380 - accuracy: 0.5630 - val_loss: 1.1507 - val_accuracy: 0.5958
Epoch 7/20
225/225 [==============================] - 35s 157ms/step - loss: 1.1831 - accuracy: 0.5814 - val_loss: 1.1623 - val_

In [84]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

6
Epoch 1/20
263/263 [==============================] - 44s 154ms/step - loss: 2.1794 - accuracy: 0.1791 - val_loss: 1.8937 - val_accuracy: 0.3097
Epoch 2/20
263/263 [==============================] - 41s 156ms/step - loss: 1.7540 - accuracy: 0.3631 - val_loss: 1.5461 - val_accuracy: 0.4512
Epoch 3/20
263/263 [==============================] - 40s 153ms/step - loss: 1.5882 - accuracy: 0.4266 - val_loss: 1.4824 - val_accuracy: 0.4813
Epoch 4/20
263/263 [==============================] - 41s 156ms/step - loss: 1.4545 - accuracy: 0.4811 - val_loss: 1.3639 - val_accuracy: 0.5167
Epoch 5/20
263/263 [==============================] - 40s 153ms/step - loss: 1.3657 - accuracy: 0.5170 - val_loss: 1.3678 - val_accuracy: 0.5138
Epoch 6/20
263/263 [==============================] - 40s 153ms/step - loss: 1.2719 - accuracy: 0.5529 - val_loss: 1.2255 - val_accuracy: 0.5677
Epoch 7/20
263/263 [==============================] - 41s 154ms/step - loss: 1.1757 - accuracy: 0.5781 - val_loss: 1.0709 - val_

In [85]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

7
Epoch 1/20
300/300 [==============================] - 49s 153ms/step - loss: 2.1508 - accuracy: 0.1928 - val_loss: 1.6953 - val_accuracy: 0.3808
Epoch 2/20
300/300 [==============================] - 48s 159ms/step - loss: 1.7458 - accuracy: 0.3615 - val_loss: 1.5886 - val_accuracy: 0.4303
Epoch 3/20
300/300 [==============================] - 50s 165ms/step - loss: 1.5576 - accuracy: 0.4375 - val_loss: 1.3055 - val_accuracy: 0.5323
Epoch 4/20
300/300 [==============================] - 50s 165ms/step - loss: 1.3916 - accuracy: 0.5049 - val_loss: 1.2621 - val_accuracy: 0.5638
Epoch 5/20
300/300 [==============================] - 50s 165ms/step - loss: 1.2761 - accuracy: 0.5447 - val_loss: 1.2228 - val_accuracy: 0.5711
Epoch 6/20
300/300 [==============================] - 50s 165ms/step - loss: 1.1971 - accuracy: 0.5810 - val_loss: 1.0751 - val_accuracy: 0.6175
Epoch 7/20
300/300 [==============================] - 50s 168ms/step - loss: 1.1311 - accuracy: 0.6061 - val_loss: 1.0346 - val_

In [86]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

8
Epoch 1/20
338/338 [==============================] - 53s 149ms/step - loss: 2.1814 - accuracy: 0.1819 - val_loss: 1.6729 - val_accuracy: 0.4033
Epoch 2/20
338/338 [==============================] - 51s 151ms/step - loss: 1.7226 - accuracy: 0.3718 - val_loss: 1.5291 - val_accuracy: 0.4537
Epoch 3/20
338/338 [==============================] - 50s 149ms/step - loss: 1.5151 - accuracy: 0.4591 - val_loss: 1.3294 - val_accuracy: 0.5434
Epoch 4/20
338/338 [==============================] - 50s 148ms/step - loss: 1.3708 - accuracy: 0.5169 - val_loss: 1.2393 - val_accuracy: 0.5639
Epoch 5/20
338/338 [==============================] - 50s 148ms/step - loss: 1.2365 - accuracy: 0.5633 - val_loss: 1.1258 - val_accuracy: 0.6023
Epoch 6/20
338/338 [==============================] - 51s 150ms/step - loss: 1.1772 - accuracy: 0.5855 - val_loss: 1.0590 - val_accuracy: 0.6280
Epoch 7/20
338/338 [==============================] - 50s 148ms/step - loss: 1.0978 - accuracy: 0.6141 - val_loss: 1.0453 - val_

In [87]:
print(n)#
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

9
Epoch 1/20
375/375 [==============================] - 61s 154ms/step - loss: 2.1545 - accuracy: 0.1901 - val_loss: 1.6878 - val_accuracy: 0.3804
Epoch 2/20
375/375 [==============================] - 55s 146ms/step - loss: 1.6938 - accuracy: 0.3870 - val_loss: 1.4150 - val_accuracy: 0.5045
Epoch 3/20
375/375 [==============================] - 55s 146ms/step - loss: 1.4873 - accuracy: 0.4693 - val_loss: 1.2903 - val_accuracy: 0.5462
Epoch 4/20
375/375 [==============================] - 54s 145ms/step - loss: 1.3293 - accuracy: 0.5310 - val_loss: 1.1587 - val_accuracy: 0.5972
Epoch 5/20
375/375 [==============================] - 55s 148ms/step - loss: 1.1948 - accuracy: 0.5804 - val_loss: 1.1170 - val_accuracy: 0.6043
Epoch 6/20
375/375 [==============================] - 59s 157ms/step - loss: 1.1349 - accuracy: 0.6054 - val_loss: 0.9990 - val_accuracy: 0.6524
Epoch 7/20
375/375 [==============================] - 59s 159ms/step - loss: 1.0523 - accuracy: 0.6336 - val_loss: 0.9852 - val_

In [89]:
n=10
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

10
Epoch 1/20
413/413 [==============================] - 58s 133ms/step - loss: 2.1456 - accuracy: 0.1934 - val_loss: 1.8038 - val_accuracy: 0.3590
Epoch 2/20
413/413 [==============================] - 54s 132ms/step - loss: 1.6533 - accuracy: 0.4027 - val_loss: 1.4783 - val_accuracy: 0.4673
Epoch 3/20
413/413 [==============================] - 54s 131ms/step - loss: 1.4394 - accuracy: 0.4833 - val_loss: 1.2495 - val_accuracy: 0.5621
Epoch 4/20
413/413 [==============================] - 54s 130ms/step - loss: 1.2949 - accuracy: 0.5396 - val_loss: 1.3029 - val_accuracy: 0.5493
Epoch 5/20
413/413 [==============================] - 54s 131ms/step - loss: 1.1708 - accuracy: 0.5890 - val_loss: 1.1727 - val_accuracy: 0.5882
Epoch 6/20
413/413 [==============================] - 54s 131ms/step - loss: 1.0846 - accuracy: 0.6242 - val_loss: 0.9590 - val_accuracy: 0.6637
Epoch 7/20
413/413 [==============================] - 54s 130ms/step - loss: 1.0336 - accuracy: 0.6433 - val_loss: 0.9625 - val

In [90]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

11
Epoch 1/20
450/450 [==============================] - 72s 152ms/step - loss: 2.0841 - accuracy: 0.2244 - val_loss: 1.6296 - val_accuracy: 0.4097
Epoch 2/20
450/450 [==============================] - 67s 149ms/step - loss: 1.6317 - accuracy: 0.4092 - val_loss: 1.3866 - val_accuracy: 0.5090
Epoch 3/20
450/450 [==============================] - 67s 149ms/step - loss: 1.4341 - accuracy: 0.4896 - val_loss: 1.2405 - val_accuracy: 0.5652
Epoch 4/20
450/450 [==============================] - 67s 149ms/step - loss: 1.2753 - accuracy: 0.5444 - val_loss: 1.1133 - val_accuracy: 0.6052
Epoch 5/20
450/450 [==============================] - 67s 149ms/step - loss: 1.1655 - accuracy: 0.5946 - val_loss: 1.0238 - val_accuracy: 0.6447
Epoch 6/20
450/450 [==============================] - 67s 149ms/step - loss: 1.0796 - accuracy: 0.6215 - val_loss: 0.9731 - val_accuracy: 0.6597
Epoch 7/20
450/450 [==============================] - 72s 161ms/step - loss: 1.0324 - accuracy: 0.6418 - val_loss: 0.9283 - val

In [91]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

12
Epoch 1/20
488/488 [==============================] - 85s 165ms/step - loss: 2.0819 - accuracy: 0.2163 - val_loss: 1.5958 - val_accuracy: 0.4290
Epoch 2/20
488/488 [==============================] - 80s 164ms/step - loss: 1.5882 - accuracy: 0.4270 - val_loss: 1.3790 - val_accuracy: 0.5045
Epoch 3/20
488/488 [==============================] - 80s 165ms/step - loss: 1.3911 - accuracy: 0.5024 - val_loss: 1.2148 - val_accuracy: 0.5717
Epoch 4/20
488/488 [==============================] - 80s 165ms/step - loss: 1.2597 - accuracy: 0.5556 - val_loss: 1.0790 - val_accuracy: 0.6174
Epoch 5/20
488/488 [==============================] - 80s 164ms/step - loss: 1.1616 - accuracy: 0.5917 - val_loss: 1.1095 - val_accuracy: 0.6177
Epoch 6/20
488/488 [==============================] - 82s 169ms/step - loss: 1.0707 - accuracy: 0.6276 - val_loss: 0.9679 - val_accuracy: 0.6570
Epoch 7/20
488/488 [==============================] - 81s 166ms/step - loss: 1.0113 - accuracy: 0.6492 - val_loss: 0.9957 - val

In [92]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

13
Epoch 1/20
525/525 [==============================] - 95s 173ms/step - loss: 2.0662 - accuracy: 0.2291 - val_loss: 1.5348 - val_accuracy: 0.4528
Epoch 2/20
525/525 [==============================] - 86s 163ms/step - loss: 1.5647 - accuracy: 0.4380 - val_loss: 1.3339 - val_accuracy: 0.5337
Epoch 3/20
525/525 [==============================] - 86s 164ms/step - loss: 1.3582 - accuracy: 0.5164 - val_loss: 1.1336 - val_accuracy: 0.5950
Epoch 4/20
525/525 [==============================] - 85s 161ms/step - loss: 1.2106 - accuracy: 0.5782 - val_loss: 1.0355 - val_accuracy: 0.6403
Epoch 5/20
525/525 [==============================] - 84s 161ms/step - loss: 1.1136 - accuracy: 0.6136 - val_loss: 0.9608 - val_accuracy: 0.6657
Epoch 6/20
525/525 [==============================] - 86s 164ms/step - loss: 1.0435 - accuracy: 0.6374 - val_loss: 1.0096 - val_accuracy: 0.6433
Epoch 7/20
525/525 [==============================] - 86s 163ms/step - loss: 0.9858 - accuracy: 0.6575 - val_loss: 0.9779 - val

In [93]:
print(n)#
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

14
Epoch 1/20
563/563 [==============================] - 99s 168ms/step - loss: 2.0471 - accuracy: 0.2396 - val_loss: 1.5133 - val_accuracy: 0.4507
Epoch 2/20
563/563 [==============================] - 92s 164ms/step - loss: 1.5228 - accuracy: 0.4486 - val_loss: 1.3542 - val_accuracy: 0.5242
Epoch 3/20
563/563 [==============================] - 93s 164ms/step - loss: 1.2997 - accuracy: 0.5399 - val_loss: 1.1437 - val_accuracy: 0.6031
Epoch 4/20
563/563 [==============================] - 93s 165ms/step - loss: 1.1716 - accuracy: 0.5926 - val_loss: 1.1031 - val_accuracy: 0.6058
Epoch 5/20
563/563 [==============================] - 92s 164ms/step - loss: 1.0746 - accuracy: 0.6275 - val_loss: 0.9891 - val_accuracy: 0.6530
Epoch 6/20
563/563 [==============================] - 92s 164ms/step - loss: 1.0059 - accuracy: 0.6521 - val_loss: 1.0559 - val_accuracy: 0.6332
Epoch 7/20
563/563 [==============================] - 92s 164ms/step - loss: 0.9660 - accuracy: 0.6650 - val_loss: 0.8646 - val

In [94]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

15
Epoch 1/20
600/600 [==============================] - 102s 163ms/step - loss: 2.0468 - accuracy: 0.2354 - val_loss: 1.5032 - val_accuracy: 0.4622
Epoch 2/20
600/600 [==============================] - 98s 163ms/step - loss: 1.5465 - accuracy: 0.4474 - val_loss: 1.3402 - val_accuracy: 0.5152
Epoch 3/20
600/600 [==============================] - 98s 163ms/step - loss: 1.3363 - accuracy: 0.5278 - val_loss: 1.1240 - val_accuracy: 0.6033
Epoch 4/20
600/600 [==============================] - 98s 163ms/step - loss: 1.1904 - accuracy: 0.5868 - val_loss: 0.9735 - val_accuracy: 0.6567
Epoch 5/20
600/600 [==============================] - 97s 162ms/step - loss: 1.0820 - accuracy: 0.6241 - val_loss: 0.9218 - val_accuracy: 0.6773
Epoch 6/20
600/600 [==============================] - 97s 163ms/step - loss: 1.0229 - accuracy: 0.6433 - val_loss: 0.9356 - val_accuracy: 0.6727
Epoch 7/20
600/600 [==============================] - 98s 163ms/step - loss: 0.9693 - accuracy: 0.6612 - val_loss: 0.8627 - va

In [95]:
time.sleep(400)

In [96]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

16
Epoch 1/20
638/638 [==============================] - 99s 148ms/step - loss: 2.0458 - accuracy: 0.2363 - val_loss: 1.5879 - val_accuracy: 0.4238
Epoch 2/20
638/638 [==============================] - 94s 148ms/step - loss: 1.5009 - accuracy: 0.4596 - val_loss: 1.3442 - val_accuracy: 0.5124
Epoch 3/20
638/638 [==============================] - 95s 148ms/step - loss: 1.2625 - accuracy: 0.5554 - val_loss: 1.0396 - val_accuracy: 0.6376
Epoch 4/20
638/638 [==============================] - 94s 148ms/step - loss: 1.1296 - accuracy: 0.6060 - val_loss: 1.1092 - val_accuracy: 0.6170
Epoch 5/20
638/638 [==============================] - 94s 148ms/step - loss: 1.0552 - accuracy: 0.6347 - val_loss: 1.0274 - val_accuracy: 0.6424
Epoch 6/20
638/638 [==============================] - 95s 149ms/step - loss: 0.9877 - accuracy: 0.6577 - val_loss: 0.8580 - val_accuracy: 0.7063
Epoch 7/20
638/638 [==============================] - 97s 152ms/step - loss: 0.9330 - accuracy: 0.6770 - val_loss: 0.8334 - val

In [97]:
time.sleep(400)

In [98]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

17
Epoch 1/20
675/675 [==============================] - 103s 147ms/step - loss: 2.0355 - accuracy: 0.2451 - val_loss: 1.4775 - val_accuracy: 0.4717
Epoch 2/20
675/675 [==============================] - 97s 144ms/step - loss: 1.4984 - accuracy: 0.4621 - val_loss: 1.2908 - val_accuracy: 0.5513
Epoch 3/20
675/675 [==============================] - 98s 145ms/step - loss: 1.2659 - accuracy: 0.5549 - val_loss: 1.1010 - val_accuracy: 0.6187
Epoch 4/20
675/675 [==============================] - 97s 144ms/step - loss: 1.1456 - accuracy: 0.5974 - val_loss: 1.0609 - val_accuracy: 0.6318
Epoch 5/20
675/675 [==============================] - 97s 144ms/step - loss: 1.0556 - accuracy: 0.6379 - val_loss: 0.9168 - val_accuracy: 0.6817
Epoch 6/20
675/675 [==============================] - 99s 147ms/step - loss: 0.9858 - accuracy: 0.6573 - val_loss: 0.8644 - val_accuracy: 0.7059
Epoch 7/20
675/675 [==============================] - 101s 149ms/step - loss: 0.9402 - accuracy: 0.6744 - val_loss: 0.8544 - v

In [99]:
time.sleep(400)

In [100]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

18
Epoch 1/20
713/713 [==============================] - 115s 153ms/step - loss: 2.0043 - accuracy: 0.2616 - val_loss: 1.5064 - val_accuracy: 0.4638
Epoch 2/20
713/713 [==============================] - 107s 151ms/step - loss: 1.4463 - accuracy: 0.4852 - val_loss: 1.1654 - val_accuracy: 0.5860
Epoch 3/20
713/713 [==============================] - 108s 152ms/step - loss: 1.2156 - accuracy: 0.5752 - val_loss: 1.1991 - val_accuracy: 0.5925
Epoch 4/20
713/713 [==============================] - 110s 154ms/step - loss: 1.0948 - accuracy: 0.6163 - val_loss: 1.0241 - val_accuracy: 0.6509
Epoch 5/20
713/713 [==============================] - 113s 158ms/step - loss: 1.0044 - accuracy: 0.6500 - val_loss: 0.8934 - val_accuracy: 0.6853
Epoch 6/20
713/713 [==============================] - 116s 163ms/step - loss: 0.9504 - accuracy: 0.6683 - val_loss: 0.8759 - val_accuracy: 0.6963
Epoch 7/20
713/713 [==============================] - 121s 170ms/step - loss: 0.9033 - accuracy: 0.6862 - val_loss: 0.926

In [ ]:
time.sleep(400)

In [101]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

19
Epoch 1/20
750/750 [==============================] - 126s 163ms/step - loss: 2.0211 - accuracy: 0.2482 - val_loss: 1.4850 - val_accuracy: 0.4836
Epoch 2/20
750/750 [==============================] - 122s 163ms/step - loss: 1.4516 - accuracy: 0.4839 - val_loss: 1.1564 - val_accuracy: 0.5902
Epoch 3/20
750/750 [==============================] - 123s 165ms/step - loss: 1.2285 - accuracy: 0.5667 - val_loss: 1.0692 - val_accuracy: 0.6255
Epoch 4/20
750/750 [==============================] - 122s 162ms/step - loss: 1.0995 - accuracy: 0.6159 - val_loss: 0.9146 - val_accuracy: 0.6831
Epoch 5/20
750/750 [==============================] - 122s 162ms/step - loss: 1.0210 - accuracy: 0.6452 - val_loss: 0.8432 - val_accuracy: 0.7132
Epoch 6/20
750/750 [==============================] - 122s 163ms/step - loss: 0.9609 - accuracy: 0.6666 - val_loss: 0.8595 - val_accuracy: 0.7082
Epoch 7/20
750/750 [==============================] - 122s 162ms/step - loss: 0.9077 - accuracy: 0.6871 - val_loss: 0.798

In [102]:
new_model_random_dir  = "D:/models/aug_22/"+dataset+"/C1/"+dataset+"_model_c1_aug_random_e1"

i=10

for model in models_random[10:]:
    model.save(new_model_random_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_random_e1_10\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_random_e1_11\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_random_e1_12\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_random_e1_13\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_random_e1_14\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_random_e1_15\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_random_e1_16\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_random_e1_17\assets
Model has been saved
INFO:tensorflow:Assets written t

In [ ]:
loading = False

models_random = []

if loading:
    for i in range(20):
        model_random_dir = "D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_random_e1_"+str(i)
        print(model_random_dir)
        model =utils.My_model('gtsrb',True,model_random_dir)
        model.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        models_random.append(model)
       

In [103]:
try:
    del random_indexes
    del top_images_by_random
    del top_labels_by_random
    del image_sets_random
    del label_sets_random
    del models_random
except:
    print("Error")

In [104]:
gc.collect()

111191

## Training guided by NC

In [105]:
# NC
nc_values = []
for i in range(1,17):
    #save_dir = "C:/Users/fjdur/Desktop/upc/project_notebooks/github_project/DL_notebooks/NC_values/nc_values_"+str(i)+".npy"
    save_dir = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/"+dataset+"_nc_values_"+str(i)+".npy"

    #print(save_dir_rand)
    tmp_values = np.load(save_dir)
    #print(tmp_values.shape)
    nc_values = np.append(nc_values,tmp_values)

In [106]:
nc_values.shape

(47999,)

In [107]:

top_images_by_nc = utils.get_x_of_indexes(list(np.flip(np.argsort(nc_values))),x_train_and_adversary)
top_labels_by_nc = utils.get_x_of_indexes(list(np.flip(np.argsort(nc_values))),y_train_and_adversary)


In [108]:
m = n_data_points
n = 0
image_sets_nc = []
label_sets_nc = []


for i in range((len(top_images_by_nc)//m)+1):
    print(i,":")
    if (i+1 >= ((len(top_images_by_nc)//m)+1)):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_nc)%m))
        top_images_by_nc_n = np.array(top_images_by_nc[:n+m+(len(top_images_by_nc)%m)])
        top_labels_by_nc_n = np.array(top_labels_by_nc[:n+m+(len(top_images_by_nc)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_nc_n = np.array(top_images_by_nc[:n+m])
        top_labels_by_nc_n = np.array(top_labels_by_nc[:n+m])
    image_sets_nc.append(top_images_by_nc_n)
    label_sets_nc.append(top_labels_by_nc_n)
    print(len(top_images_by_nc_n))
    n += m




0 :
0  ->  2400
2400
1 :
0  ->  4800
4800
2 :
0  ->  7200
7200
3 :
0  ->  9600
9600
4 :
0  ->  12000
12000
5 :
0  ->  14400
14400
6 :
0  ->  16800
16800
7 :
0  ->  19200
19200
8 :
0  ->  21600
21600
9 :
0  ->  24000
24000
10 :
0  ->  26400
26400
11 :
0  ->  28800
28800
12 :
0  ->  31200
31200
13 :
0  ->  33600
33600
14 :
0  ->  36000
36000
15 :
0  ->  38400
38400
16 :
0  ->  40800
40800
17 :
0  ->  43200
43200
18 :
0  ->  45600
45600
19 :
Last
0  ->  50399
47999


In [109]:
print(model_dir)

models_nc = []
for i in range(len(label_sets_nc)):
    print(i,":")
    model = utils.My_model(dataset,False,None)
    model.compile_model()
    models_nc.append(model)


C:/Users/fjdur/Documents/upc-july/models/model_02
0 :
Model compiled
1 :
Model compiled
2 :
Model compiled
3 :
Model compiled
4 :
Model compiled
5 :
Model compiled
6 :
Model compiled
7 :
Model compiled
8 :
Model compiled
9 :
Model compiled
10 :
Model compiled
11 :
Model compiled
12 :
Model compiled
13 :
Model compiled
14 :
Model compiled
15 :
Model compiled
16 :
Model compiled
17 :
Model compiled
18 :
Model compiled
19 :
Model compiled


In [110]:
n=0
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

0
Epoch 1/20
38/38 [==============================] - 16s 317ms/step - loss: 1.9051 - accuracy: 0.2714 - val_loss: 2.4005 - val_accuracy: 0.1757
Epoch 2/20
38/38 [==============================] - 12s 332ms/step - loss: 1.5726 - accuracy: 0.4711 - val_loss: 2.5270 - val_accuracy: 0.1973
Epoch 3/20
38/38 [==============================] - 12s 326ms/step - loss: 1.3011 - accuracy: 0.5880 - val_loss: 2.7796 - val_accuracy: 0.2498
Epoch 4/20
38/38 [==============================] - 12s 325ms/step - loss: 1.1736 - accuracy: 0.6286 - val_loss: 2.2224 - val_accuracy: 0.2725
Epoch 5/20
38/38 [==============================] - 12s 324ms/step - loss: 1.0127 - accuracy: 0.6825 - val_loss: 2.9630 - val_accuracy: 0.2560
Epoch 6/20
38/38 [==============================] - 12s 324ms/step - loss: 0.9543 - accuracy: 0.6936 - val_loss: 2.2280 - val_accuracy: 0.2998
Epoch 7/20
38/38 [==============================] - 12s 324ms/step - loss: 0.8049 - accuracy: 0.7557 - val_loss: 2.4570 - val_accuracy: 0.32

In [111]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

1
Epoch 1/20
75/75 [==============================] - 23s 254ms/step - loss: 1.9840 - accuracy: 0.2377 - val_loss: 2.8770 - val_accuracy: 0.1916
Epoch 2/20
75/75 [==============================] - 18s 241ms/step - loss: 1.4696 - accuracy: 0.5283 - val_loss: 2.3613 - val_accuracy: 0.2498
Epoch 3/20
75/75 [==============================] - 18s 240ms/step - loss: 1.2151 - accuracy: 0.6276 - val_loss: 2.5460 - val_accuracy: 0.2348
Epoch 4/20
75/75 [==============================] - 18s 240ms/step - loss: 1.1060 - accuracy: 0.6597 - val_loss: 2.1658 - val_accuracy: 0.3321
Epoch 5/20
75/75 [==============================] - 18s 239ms/step - loss: 0.9264 - accuracy: 0.7091 - val_loss: 2.3202 - val_accuracy: 0.3228
Epoch 6/20
75/75 [==============================] - 18s 239ms/step - loss: 0.8102 - accuracy: 0.7465 - val_loss: 2.1588 - val_accuracy: 0.3625
Epoch 7/20
75/75 [==============================] - 18s 240ms/step - loss: 0.7491 - accuracy: 0.7658 - val_loss: 1.9342 - val_accuracy: 0.35

In [112]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

2
Epoch 1/20
113/113 [==============================] - 28s 215ms/step - loss: 2.0302 - accuracy: 0.2642 - val_loss: 2.3569 - val_accuracy: 0.2003
Epoch 2/20
113/113 [==============================] - 24s 216ms/step - loss: 1.4686 - accuracy: 0.5289 - val_loss: 2.4323 - val_accuracy: 0.2990
Epoch 3/20
113/113 [==============================] - 24s 214ms/step - loss: 1.1621 - accuracy: 0.6324 - val_loss: 2.0864 - val_accuracy: 0.3341
Epoch 4/20
113/113 [==============================] - 24s 214ms/step - loss: 0.9889 - accuracy: 0.6954 - val_loss: 1.9016 - val_accuracy: 0.3787
Epoch 5/20
113/113 [==============================] - 25s 218ms/step - loss: 0.8202 - accuracy: 0.7420 - val_loss: 1.8357 - val_accuracy: 0.4117
Epoch 6/20
113/113 [==============================] - 24s 216ms/step - loss: 0.7238 - accuracy: 0.7687 - val_loss: 1.7527 - val_accuracy: 0.4391
Epoch 7/20
113/113 [==============================] - 24s 215ms/step - loss: 0.6588 - accuracy: 0.7951 - val_loss: 1.7835 - val_

In [113]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

3
Epoch 1/20
150/150 [==============================] - 36s 213ms/step - loss: 1.9928 - accuracy: 0.2758 - val_loss: 2.1798 - val_accuracy: 0.2772
Epoch 2/20
150/150 [==============================] - 30s 199ms/step - loss: 1.3838 - accuracy: 0.5602 - val_loss: 2.2969 - val_accuracy: 0.3002
Epoch 3/20
150/150 [==============================] - 29s 197ms/step - loss: 1.1568 - accuracy: 0.6364 - val_loss: 1.8948 - val_accuracy: 0.3804
Epoch 4/20
150/150 [==============================] - 30s 197ms/step - loss: 0.9950 - accuracy: 0.6862 - val_loss: 1.7080 - val_accuracy: 0.4277
Epoch 5/20
150/150 [==============================] - 29s 197ms/step - loss: 0.8576 - accuracy: 0.7287 - val_loss: 1.7731 - val_accuracy: 0.4182
Epoch 6/20
150/150 [==============================] - 29s 196ms/step - loss: 0.7966 - accuracy: 0.7507 - val_loss: 1.5911 - val_accuracy: 0.4768
Epoch 7/20
150/150 [==============================] - 29s 197ms/step - loss: 0.7047 - accuracy: 0.7803 - val_loss: 1.5284 - val_

In [114]:
print(n)#
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

4
Epoch 1/20
188/188 [==============================] - 40s 192ms/step - loss: 2.0101 - accuracy: 0.2837 - val_loss: 2.1466 - val_accuracy: 0.2798
Epoch 2/20
188/188 [==============================] - 36s 192ms/step - loss: 1.4212 - accuracy: 0.5467 - val_loss: 1.9580 - val_accuracy: 0.3543
Epoch 3/20
188/188 [==============================] - 36s 190ms/step - loss: 1.1504 - accuracy: 0.6291 - val_loss: 1.7249 - val_accuracy: 0.4237
Epoch 4/20
188/188 [==============================] - 36s 192ms/step - loss: 0.9954 - accuracy: 0.6773 - val_loss: 1.8431 - val_accuracy: 0.4212
Epoch 5/20
188/188 [==============================] - 36s 190ms/step - loss: 0.8627 - accuracy: 0.7253 - val_loss: 1.6782 - val_accuracy: 0.4639
Epoch 6/20
188/188 [==============================] - 36s 189ms/step - loss: 0.7744 - accuracy: 0.7537 - val_loss: 1.6277 - val_accuracy: 0.4793
Epoch 7/20
188/188 [==============================] - 36s 192ms/step - loss: 0.7021 - accuracy: 0.7738 - val_loss: 1.4267 - val_

In [115]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

5
Epoch 1/20
225/225 [==============================] - 45s 183ms/step - loss: 1.9934 - accuracy: 0.2882 - val_loss: 1.9465 - val_accuracy: 0.3180
Epoch 2/20
225/225 [==============================] - 41s 183ms/step - loss: 1.4032 - accuracy: 0.5370 - val_loss: 1.6915 - val_accuracy: 0.3996
Epoch 3/20
225/225 [==============================] - 41s 183ms/step - loss: 1.1596 - accuracy: 0.6119 - val_loss: 1.5859 - val_accuracy: 0.4540
Epoch 4/20
225/225 [==============================] - 41s 182ms/step - loss: 0.9448 - accuracy: 0.6948 - val_loss: 1.4373 - val_accuracy: 0.5010
Epoch 5/20
225/225 [==============================] - 41s 181ms/step - loss: 0.8400 - accuracy: 0.7282 - val_loss: 1.3865 - val_accuracy: 0.5110
Epoch 6/20
225/225 [==============================] - 41s 182ms/step - loss: 0.7668 - accuracy: 0.7486 - val_loss: 1.3375 - val_accuracy: 0.5315
Epoch 7/20
225/225 [==============================] - 41s 182ms/step - loss: 0.7036 - accuracy: 0.7657 - val_loss: 1.3475 - val_

In [116]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

6
Epoch 1/20
263/263 [==============================] - 51s 177ms/step - loss: 2.0276 - accuracy: 0.2811 - val_loss: 1.8781 - val_accuracy: 0.3277
Epoch 2/20
263/263 [==============================] - 46s 177ms/step - loss: 1.4218 - accuracy: 0.5268 - val_loss: 1.7560 - val_accuracy: 0.3839
Epoch 3/20
263/263 [==============================] - 46s 176ms/step - loss: 1.1709 - accuracy: 0.6136 - val_loss: 1.5400 - val_accuracy: 0.4496
Epoch 4/20
263/263 [==============================] - 46s 176ms/step - loss: 0.9846 - accuracy: 0.6716 - val_loss: 1.5052 - val_accuracy: 0.4762
Epoch 5/20
263/263 [==============================] - 46s 176ms/step - loss: 0.8612 - accuracy: 0.7170 - val_loss: 1.4177 - val_accuracy: 0.5105
Epoch 6/20
263/263 [==============================] - 46s 177ms/step - loss: 0.7819 - accuracy: 0.7441 - val_loss: 1.2993 - val_accuracy: 0.5539
Epoch 7/20
263/263 [==============================] - 47s 177ms/step - loss: 0.7224 - accuracy: 0.7610 - val_loss: 1.2475 - val_

In [117]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

7
Epoch 1/20
300/300 [==============================] - 56s 174ms/step - loss: 1.9951 - accuracy: 0.2870 - val_loss: 1.8925 - val_accuracy: 0.3225
Epoch 2/20
300/300 [==============================] - 51s 170ms/step - loss: 1.3939 - accuracy: 0.5306 - val_loss: 1.6203 - val_accuracy: 0.4381
Epoch 3/20
300/300 [==============================] - 52s 172ms/step - loss: 1.1682 - accuracy: 0.6082 - val_loss: 1.4850 - val_accuracy: 0.4887
Epoch 4/20
300/300 [==============================] - 51s 171ms/step - loss: 0.9607 - accuracy: 0.6786 - val_loss: 1.3082 - val_accuracy: 0.5538
Epoch 5/20
300/300 [==============================] - 51s 170ms/step - loss: 0.8710 - accuracy: 0.7116 - val_loss: 1.3428 - val_accuracy: 0.5393
Epoch 6/20
300/300 [==============================] - 51s 171ms/step - loss: 0.7993 - accuracy: 0.7354 - val_loss: 1.2095 - val_accuracy: 0.5775
Epoch 7/20
300/300 [==============================] - 51s 171ms/step - loss: 0.7502 - accuracy: 0.7528 - val_loss: 1.2126 - val_

In [118]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

8
Epoch 1/20
338/338 [==============================] - 62s 170ms/step - loss: 1.9828 - accuracy: 0.2885 - val_loss: 1.8759 - val_accuracy: 0.3421
Epoch 2/20
338/338 [==============================] - 58s 171ms/step - loss: 1.3845 - accuracy: 0.5286 - val_loss: 1.5866 - val_accuracy: 0.4497
Epoch 3/20
338/338 [==============================] - 58s 171ms/step - loss: 1.1269 - accuracy: 0.6165 - val_loss: 1.6878 - val_accuracy: 0.4415
Epoch 4/20
338/338 [==============================] - 58s 170ms/step - loss: 0.9983 - accuracy: 0.6659 - val_loss: 1.4412 - val_accuracy: 0.5118
Epoch 5/20
338/338 [==============================] - 58s 171ms/step - loss: 0.8885 - accuracy: 0.7020 - val_loss: 1.2489 - val_accuracy: 0.5738
Epoch 6/20
338/338 [==============================] - 58s 170ms/step - loss: 0.8151 - accuracy: 0.7259 - val_loss: 1.1986 - val_accuracy: 0.5832
Epoch 7/20
338/338 [==============================] - 57s 170ms/step - loss: 0.7469 - accuracy: 0.7482 - val_loss: 1.1044 - val_

In [119]:
print(n)#
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

9
Epoch 1/20
375/375 [==============================] - 60s 151ms/step - loss: 2.0727 - accuracy: 0.2488 - val_loss: 1.7097 - val_accuracy: 0.3902
Epoch 2/20
375/375 [==============================] - 56s 150ms/step - loss: 1.4489 - accuracy: 0.4972 - val_loss: 1.4595 - val_accuracy: 0.4723
Epoch 3/20
375/375 [==============================] - 56s 149ms/step - loss: 1.1736 - accuracy: 0.6005 - val_loss: 1.4768 - val_accuracy: 0.4810
Epoch 4/20
375/375 [==============================] - 56s 149ms/step - loss: 1.0273 - accuracy: 0.6502 - val_loss: 1.1866 - val_accuracy: 0.5847
Epoch 5/20
375/375 [==============================] - 56s 149ms/step - loss: 0.9099 - accuracy: 0.6956 - val_loss: 1.1231 - val_accuracy: 0.6065
Epoch 6/20
375/375 [==============================] - 56s 149ms/step - loss: 0.8338 - accuracy: 0.7186 - val_loss: 1.1511 - val_accuracy: 0.6032
Epoch 7/20
375/375 [==============================] - 56s 148ms/step - loss: 0.7769 - accuracy: 0.7332 - val_loss: 1.1780 - val_

In [121]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

10
Epoch 1/20
413/413 [==============================] - 57s 131ms/step - loss: 1.9805 - accuracy: 0.2837 - val_loss: 1.6373 - val_accuracy: 0.4082
Epoch 2/20
413/413 [==============================] - 54s 130ms/step - loss: 1.4218 - accuracy: 0.5015 - val_loss: 1.5450 - val_accuracy: 0.4473
Epoch 3/20
413/413 [==============================] - 54s 130ms/step - loss: 1.1940 - accuracy: 0.5918 - val_loss: 1.3420 - val_accuracy: 0.5113
Epoch 4/20
413/413 [==============================] - 54s 130ms/step - loss: 1.0354 - accuracy: 0.6472 - val_loss: 1.1803 - val_accuracy: 0.5843
Epoch 5/20
413/413 [==============================] - 54s 130ms/step - loss: 0.9199 - accuracy: 0.6908 - val_loss: 1.1345 - val_accuracy: 0.6145
Epoch 6/20
413/413 [==============================] - 54s 131ms/step - loss: 0.8464 - accuracy: 0.7153 - val_loss: 1.1264 - val_accuracy: 0.6163
Epoch 7/20
413/413 [==============================] - 54s 132ms/step - loss: 0.7898 - accuracy: 0.7330 - val_loss: 1.0161 - val

In [122]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

11
Epoch 1/20
450/450 [==============================] - 70s 149ms/step - loss: 2.0309 - accuracy: 0.2599 - val_loss: 1.6056 - val_accuracy: 0.4247
Epoch 2/20
450/450 [==============================] - 67s 148ms/step - loss: 1.4333 - accuracy: 0.5000 - val_loss: 1.4128 - val_accuracy: 0.5000
Epoch 3/20
450/450 [==============================] - 66s 147ms/step - loss: 1.2001 - accuracy: 0.5866 - val_loss: 1.2803 - val_accuracy: 0.5411
Epoch 4/20
450/450 [==============================] - 67s 148ms/step - loss: 1.0447 - accuracy: 0.6398 - val_loss: 1.1562 - val_accuracy: 0.6002
Epoch 5/20
450/450 [==============================] - 67s 149ms/step - loss: 0.9239 - accuracy: 0.6863 - val_loss: 1.0639 - val_accuracy: 0.6322
Epoch 6/20
450/450 [==============================] - 75s 167ms/step - loss: 0.8689 - accuracy: 0.7048 - val_loss: 1.0337 - val_accuracy: 0.6373
Epoch 7/20
450/450 [==============================] - 75s 167ms/step - loss: 0.8046 - accuracy: 0.7225 - val_loss: 1.0296 - val

In [123]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

12
Epoch 1/20
488/488 [==============================] - 86s 167ms/step - loss: 2.0031 - accuracy: 0.2747 - val_loss: 1.6663 - val_accuracy: 0.3984
Epoch 2/20
488/488 [==============================] - 82s 167ms/step - loss: 1.3700 - accuracy: 0.5234 - val_loss: 1.3244 - val_accuracy: 0.5271
Epoch 3/20
488/488 [==============================] - 83s 170ms/step - loss: 1.1256 - accuracy: 0.6109 - val_loss: 1.2897 - val_accuracy: 0.5641
Epoch 4/20
488/488 [==============================] - 82s 168ms/step - loss: 0.9936 - accuracy: 0.6578 - val_loss: 1.1523 - val_accuracy: 0.5996
Epoch 5/20
488/488 [==============================] - 82s 168ms/step - loss: 0.9100 - accuracy: 0.6924 - val_loss: 1.1322 - val_accuracy: 0.6122
Epoch 6/20
488/488 [==============================] - 82s 167ms/step - loss: 0.8390 - accuracy: 0.7142 - val_loss: 0.9861 - val_accuracy: 0.6553
Epoch 7/20
488/488 [==============================] - 82s 168ms/step - loss: 0.7945 - accuracy: 0.7288 - val_loss: 0.9997 - val

In [124]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

13
Epoch 1/20
525/525 [==============================] - 82s 150ms/step - loss: 2.0037 - accuracy: 0.2686 - val_loss: 1.5837 - val_accuracy: 0.4283
Epoch 2/20
525/525 [==============================] - 78s 149ms/step - loss: 1.4355 - accuracy: 0.4942 - val_loss: 1.2960 - val_accuracy: 0.5433
Epoch 3/20
525/525 [==============================] - 80s 152ms/step - loss: 1.1988 - accuracy: 0.5830 - val_loss: 1.2772 - val_accuracy: 0.5606
Epoch 4/20
525/525 [==============================] - 81s 155ms/step - loss: 1.0506 - accuracy: 0.6377 - val_loss: 1.0547 - val_accuracy: 0.6376
Epoch 5/20
525/525 [==============================] - 78s 149ms/step - loss: 0.9500 - accuracy: 0.6761 - val_loss: 1.0207 - val_accuracy: 0.6399
Epoch 6/20
525/525 [==============================] - 78s 149ms/step - loss: 0.8930 - accuracy: 0.6951 - val_loss: 0.9456 - val_accuracy: 0.6699
Epoch 7/20
525/525 [==============================] - 79s 151ms/step - loss: 0.8238 - accuracy: 0.7173 - val_loss: 0.9492 - val

In [125]:
print(n)#
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

14
Epoch 1/20
563/563 [==============================] - 101s 173ms/step - loss: 2.0144 - accuracy: 0.2560 - val_loss: 1.6627 - val_accuracy: 0.4092
Epoch 2/20
563/563 [==============================] - 97s 173ms/step - loss: 1.4158 - accuracy: 0.4952 - val_loss: 1.2557 - val_accuracy: 0.5529
Epoch 3/20
563/563 [==============================] - 97s 172ms/step - loss: 1.1690 - accuracy: 0.5956 - val_loss: 1.1749 - val_accuracy: 0.5914
Epoch 4/20
563/563 [==============================] - 97s 172ms/step - loss: 1.0332 - accuracy: 0.6435 - val_loss: 1.0538 - val_accuracy: 0.6325
Epoch 5/20
563/563 [==============================] - 97s 173ms/step - loss: 0.9509 - accuracy: 0.6730 - val_loss: 1.0146 - val_accuracy: 0.6453
Epoch 6/20
563/563 [==============================] - 97s 172ms/step - loss: 0.8869 - accuracy: 0.6955 - val_loss: 0.9287 - val_accuracy: 0.6811
Epoch 7/20
563/563 [==============================] - 96s 171ms/step - loss: 0.8318 - accuracy: 0.7123 - val_loss: 0.9055 - va

In [126]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

15
Epoch 1/20
600/600 [==============================] - 107s 171ms/step - loss: 2.0017 - accuracy: 0.2702 - val_loss: 1.5054 - val_accuracy: 0.4473
Epoch 2/20
600/600 [==============================] - 102s 171ms/step - loss: 1.4084 - accuracy: 0.4968 - val_loss: 1.2084 - val_accuracy: 0.5749
Epoch 3/20
600/600 [==============================] - 102s 170ms/step - loss: 1.1819 - accuracy: 0.5859 - val_loss: 1.0846 - val_accuracy: 0.6250
Epoch 4/20
600/600 [==============================] - 102s 170ms/step - loss: 1.0386 - accuracy: 0.6414 - val_loss: 0.9860 - val_accuracy: 0.6523
Epoch 5/20
600/600 [==============================] - 102s 170ms/step - loss: 0.9575 - accuracy: 0.6665 - val_loss: 0.9177 - val_accuracy: 0.6811
Epoch 6/20
600/600 [==============================] - 102s 170ms/step - loss: 0.8926 - accuracy: 0.6905 - val_loss: 0.9954 - val_accuracy: 0.6516
Epoch 7/20
600/600 [==============================] - 102s 170ms/step - loss: 0.8528 - accuracy: 0.7035 - val_loss: 0.897

In [127]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

16
Epoch 1/20
638/638 [==============================] - 115s 173ms/step - loss: 2.0170 - accuracy: 0.2597 - val_loss: 1.5187 - val_accuracy: 0.4540
Epoch 2/20
638/638 [==============================] - 109s 171ms/step - loss: 1.4540 - accuracy: 0.4801 - val_loss: 1.2471 - val_accuracy: 0.5571
Epoch 3/20
638/638 [==============================] - 109s 171ms/step - loss: 1.2071 - accuracy: 0.5795 - val_loss: 1.0628 - val_accuracy: 0.6345
Epoch 4/20
638/638 [==============================] - 109s 171ms/step - loss: 1.0575 - accuracy: 0.6348 - val_loss: 1.0439 - val_accuracy: 0.6398
Epoch 5/20
638/638 [==============================] - 109s 171ms/step - loss: 0.9774 - accuracy: 0.6604 - val_loss: 1.1301 - val_accuracy: 0.6019
Epoch 6/20
638/638 [==============================] - 109s 171ms/step - loss: 0.9227 - accuracy: 0.6804 - val_loss: 0.8956 - val_accuracy: 0.6884
Epoch 7/20
638/638 [==============================] - 109s 171ms/step - loss: 0.8577 - accuracy: 0.7030 - val_loss: 0.910

In [ ]:
time.sleep(400)

In [128]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

17
Epoch 1/20
675/675 [==============================] - 118s 169ms/step - loss: 1.9835 - accuracy: 0.2652 - val_loss: 1.4402 - val_accuracy: 0.4896
Epoch 2/20
675/675 [==============================] - 114s 168ms/step - loss: 1.3986 - accuracy: 0.5054 - val_loss: 1.1751 - val_accuracy: 0.5949
Epoch 3/20
675/675 [==============================] - 114s 169ms/step - loss: 1.1792 - accuracy: 0.5929 - val_loss: 1.0578 - val_accuracy: 0.6310
Epoch 4/20
675/675 [==============================] - 114s 168ms/step - loss: 1.0447 - accuracy: 0.6403 - val_loss: 1.2224 - val_accuracy: 0.5679
Epoch 5/20
675/675 [==============================] - 114s 168ms/step - loss: 0.9775 - accuracy: 0.6617 - val_loss: 0.9085 - val_accuracy: 0.6877
Epoch 6/20
675/675 [==============================] - 114s 168ms/step - loss: 0.9117 - accuracy: 0.6826 - val_loss: 0.9366 - val_accuracy: 0.6732
Epoch 7/20
675/675 [==============================] - 113s 168ms/step - loss: 0.8644 - accuracy: 0.7009 - val_loss: 0.820

In [ ]:
time.sleep(400)

In [129]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

18
Epoch 1/20
713/713 [==============================] - 113s 154ms/step - loss: 1.9891 - accuracy: 0.2579 - val_loss: 1.5708 - val_accuracy: 0.4333
Epoch 2/20
713/713 [==============================] - 109s 153ms/step - loss: 1.5035 - accuracy: 0.4620 - val_loss: 1.3627 - val_accuracy: 0.5018
Epoch 3/20
713/713 [==============================] - 110s 154ms/step - loss: 1.2366 - accuracy: 0.5640 - val_loss: 1.1073 - val_accuracy: 0.6067
Epoch 4/20
713/713 [==============================] - 110s 154ms/step - loss: 1.0962 - accuracy: 0.6155 - val_loss: 0.9945 - val_accuracy: 0.6577
Epoch 5/20
713/713 [==============================] - 110s 154ms/step - loss: 1.0109 - accuracy: 0.6472 - val_loss: 0.9237 - val_accuracy: 0.6780
Epoch 6/20
713/713 [==============================] - 110s 154ms/step - loss: 0.9473 - accuracy: 0.6742 - val_loss: 0.8969 - val_accuracy: 0.6868
Epoch 7/20
713/713 [==============================] - 109s 153ms/step - loss: 0.8992 - accuracy: 0.6905 - val_loss: 0.919

In [ ]:
time.sleep(400)

In [130]:
print(n)#
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val,epochs=20,batch_size = 64)
n = n+1

19
Epoch 1/20
750/750 [==============================] - 135s 174ms/step - loss: 2.0215 - accuracy: 0.2509 - val_loss: 1.4059 - val_accuracy: 0.4942
Epoch 2/20
750/750 [==============================] - 131s 175ms/step - loss: 1.4542 - accuracy: 0.4791 - val_loss: 1.1537 - val_accuracy: 0.5933
Epoch 3/20
750/750 [==============================] - 131s 175ms/step - loss: 1.2189 - accuracy: 0.5713 - val_loss: 1.0570 - val_accuracy: 0.6319
Epoch 4/20
750/750 [==============================] - 131s 175ms/step - loss: 1.0843 - accuracy: 0.6263 - val_loss: 0.8826 - val_accuracy: 0.6986
Epoch 5/20
750/750 [==============================] - 131s 175ms/step - loss: 1.0050 - accuracy: 0.6511 - val_loss: 0.8826 - val_accuracy: 0.6994
Epoch 6/20
750/750 [==============================] - 131s 175ms/step - loss: 0.9373 - accuracy: 0.6731 - val_loss: 0.8130 - val_accuracy: 0.7199
Epoch 7/20
750/750 [==============================] - 132s 176ms/step - loss: 0.8941 - accuracy: 0.6916 - val_loss: 0.774

In [131]:
new_model_nc_dir  = "D:/models/aug_22/"+dataset+"/C1/"+dataset+"_model_c1_aug_nc_e1"

i=10

for model in models_nc[10:]:
    model.save(new_model_nc_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_nc_e1_10\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_nc_e1_11\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_nc_e1_12\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_nc_e1_13\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_nc_e1_14\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_nc_e1_15\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_nc_e1_16\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cifar_model_c1_aug_nc_e1_17\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/cifar/C1/cif

In [ ]:
loading = False

models_nc = []

if loading:
    for i in range(20):
        model_nc_dir = "D:/models/aug_22/gtsrb/C1/gtsrb_model_c1_aug_nc_e1"+str(i)
        print(model_nc_dir)
        model =utils.My_model(dataset,True,model_nc_dir)
        model.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
        models_nc.append(model)
       

In [ ]:
try:
    del nc_values
    del top_images_by_nc
    del top_labels_by_nc
    del image_sets_nc
    del label_sets_nc
    del models_nc
except:
    print("Error")

In [ ]:
gc.collect()

## Evaluating

In [ ]:
model_original.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [ ]:
# Metrics using adversarial test
evaluate_lsa_0 = []
evaluate_dsa_0 = []
evaluate_nc_0 = []
evaluate_random_0 = []

evaluate_lsa_0.append(model_original.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
evaluate_dsa_0.append(model_original.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
evaluate_nc_0.append(model_original.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
evaluate_random_0.append(model_original.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))

for model in models_lsa:
    evaluate_lsa_0.append(model.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
    
for model in models_dsa:
    evaluate_dsa_0.append(model.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
    
for model in models_random:
    evaluate_random_0.append(model.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
    
for model in models_nc:
    evaluate_nc_0.append(model.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))

In [ ]:
evaluate_lsa_0[0]=[0,0,0,0]
evaluate_dsa_0[0]=[0,0,0,0]
evaluate_random_0[0]=[0,0,0,0]
evaluate_nc_0[0]=[0,0,0,0]

In [ ]:
print(len(evaluate_lsa_0))
print(len(evaluate_dsa_0))
print(len(evaluate_random_0))
print(len(evaluate_nc_0))

In [ ]:
# Metrics using original test
evaluate_lsa_1 = []
evaluate_dsa_1 = []
evaluate_nc_1 = []
evaluate_random_1 = []

evaluate_lsa_1.append(model_original.evaluate(x_test,y_test))
evaluate_dsa_1.append(model_original.evaluate(x_test,y_test))
evaluate_nc_1.append(model_original.evaluate(x_test,y_test))
evaluate_random_1.append(model_original.evaluate(x_test,y_test))
print("lsa---")
for model in models_lsa:
    evaluate_lsa_1.append(model.evaluate(x_test,y_test))

print("dsa---")
    
for model in models_dsa:
    evaluate_dsa_1.append(model.evaluate(x_test,y_test))
    

print("random---")

for model in models_random:
    evaluate_random_1.append(model.evaluate(x_test,y_test))
    
print("nc---")

for model in models_nc:
    evaluate_nc_1.append(model.evaluate(x_test,y_test))


In [ ]:
evaluate_lsa_1[0]=[0,0,0,0]
evaluate_dsa_1[0]=[0,0,0,0]
evaluate_random_1[0]=[0,0,0,0]
evaluate_nc_1[0]=[0,0,0,0]

In [ ]:

print(len(evaluate_lsa_1))
print(len(evaluate_dsa_1))
print(len(evaluate_random_1))
print(len(evaluate_nc_1))

In [ ]:
type(evaluate_dsa_0[0][0])

In [ ]:
import pandas as pd

df_evaluate_lsa_0 = pd.DataFrame(np.array(evaluate_lsa_0),columns=["loss","accuracy","precision","recall"])
df_evaluate_lsa_1 = pd.DataFrame(np.array(evaluate_lsa_1),columns=["loss","accuracy","precision","recall"])
df_evaluate_dsa_0 = pd.DataFrame(np.array(evaluate_dsa_0),columns=["loss","accuracy","precision","recall"])
df_evaluate_dsa_1 = pd.DataFrame(np.array(evaluate_dsa_1),columns=["loss","accuracy","precision","recall"])

df_evaluate_random_0 = pd.DataFrame(np.array(evaluate_random_0),columns=["loss","accuracy","precision","recall"])
df_evaluate_random_1 = pd.DataFrame(np.array(evaluate_random_1),columns=["loss","accuracy","precision","recall"])
df_evaluate_nc_0 = pd.DataFrame(np.array(evaluate_nc_0),columns=["loss","accuracy","precision","recall"])
df_evaluate_nc_1 = pd.DataFrame(np.array(evaluate_nc_1),columns=["loss","accuracy","precision","recall"])


In [ ]:
# Original test set + adversarial set

metric ="accuracy"
accuracy_lsa_3 = (np.array(df_evaluate_lsa_0[metric])+np.array(df_evaluate_lsa_1[metric]))/2
accuracy_dsa_3 = (np.array(df_evaluate_dsa_0[metric])+np.array(df_evaluate_dsa_1[metric]))/2
accuracy_nc_3 = (np.array(df_evaluate_nc_0[metric])+np.array(df_evaluate_nc_1[metric]))/2
accuracy_random_3 = (np.array(df_evaluate_random_0[metric])+np.array(df_evaluate_random_1[metric]))/2


## Charts

In [ ]:
n_inputs = [1800*i for i in range(20)]
n_inputs.append(len(x_train_and_adversary))

In [ ]:
print(n_inputs)

In [ ]:
cd "C:/Users/fjdur/Desktop/gtsrb_graphs/e2_graphs_dec/"

In [ ]:
linestyles = ['solid','dotted','dashed','dashdot']
colors =['k','k','k','k']

In [ ]:
#configuration 1
"""
6) Incremental guided retraining starting from the original model using only the new adversarial inputs.
Incremental training, starting with the previous trained model. Using at each iteration a subset of the new inputs.
"""
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

#metric = "accuracy" # accuracy loss
my_metrics =["accuracy","loss","precision","recall"]

for metric in my_metrics:
    plt.clf()
    plt.plot(n_inputs,df_evaluate_lsa_0[metric],colors[0],linestyle=linestyles[0])
    plt.plot(n_inputs,df_evaluate_dsa_0[metric],colors[1],linestyle=linestyles[1])
    plt.plot(n_inputs,df_evaluate_random_0[metric],colors[2],linestyle=linestyles[2])
    plt.plot(n_inputs,df_evaluate_nc_0[metric],colors[3],linestyle=linestyles[3])


    legend_elements = [Line2D([0], [0], color='k', label='LSA',ls = linestyles[0]),
                   Line2D([0], [0], color='k', label='DSA',ls = linestyles[1]),
                   Line2D([0], [0], color='k', label='Random',ls = linestyles[2]),
                   Line2D([0], [0], color='k', label='NC',ls = linestyles[3])]



    plt.legend(handles=legend_elements)#
    plt.title(metric + " with test set of adversarial examples FGSM")
    plt.xlim([0, 37000])
    #plt.ylim([0, 1])
    plt.xlabel('number of inputs')
    plt.ylabel(metric)
    plt.savefig("gtsrb_c1_"+metric + "_0.svg")
    plt.show()

In [ ]:
#configuration 1
"""
6) Incremental guided retraining starting from the original model using only the new adversarial inputs.
Incremental training, starting with the previous trained model. Using at each iteration a subset of the new inputs.
"""
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

my_metrics =["accuracy","loss","precision","recall"]

for metric in my_metrics:
    
#metric = "accuracy" # accuracy loss
    plt.clf()
    plt.plot(n_inputs,df_evaluate_lsa_1[metric],colors[0],linestyle=linestyles[0])
    plt.plot(n_inputs,df_evaluate_dsa_1[metric],colors[1],linestyle=linestyles[1])
    plt.plot(n_inputs,df_evaluate_random_1[metric],colors[2],linestyle=linestyles[2])
    plt.plot(n_inputs,df_evaluate_nc_1[metric],colors[3],linestyle=linestyles[3])

    legend_elements = [Line2D([0], [0], color='k', label='LSA',ls = linestyles[0]),
                   Line2D([0], [0], color='k', label='DSA',ls = linestyles[1]),
                   Line2D([0], [0], color='k', label='Random',ls = linestyles[2]),
                   Line2D([0], [0], color='k', label='NC',ls = linestyles[3])]



    plt.legend(handles=legend_elements)#
    plt.title(metric + " with original set")
    plt.xlim([0, 37000])
    #plt.ylim([0, 1])
    plt.xlabel('number of inputs')
    plt.ylabel(metric)
    plt.savefig("gtsrb_c1_"+metric + "_1.svg")
    plt.show()

In [ ]:
#adversarial jsma test set
#configuration 3
"""
6) Incremental guided retraining starting from the original model using only the new adversarial inputs.
Incremental training, starting with the previous trained model. Using at each iteration a subset of the new inputs.
"""
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

plt.plot(n_inputs,accuracy_lsa_3,colors[0],linestyle=linestyles[0])
plt.plot(n_inputs,accuracy_dsa_3,colors[1],linestyle=linestyles[1])
plt.plot(n_inputs,accuracy_random_3,colors[2],linestyle=linestyles[2])
plt.plot(n_inputs,accuracy_nc_3,colors[3],linestyle=linestyles[3])


legend_elements = [Line2D([0], [0], color='k', label='LSA',ls = linestyles[0]),
                   Line2D([0], [0], color='k', label='DSA',ls = linestyles[1]),
                   Line2D([0], [0], color='k', label='Random',ls = linestyles[2]),
                   Line2D([0], [0], color='k', label='NC',ls = linestyles[3])]



plt.legend(handles=legend_elements)#
plt.title("accuracy with both sets")

print(n_inputs[np.argmax(accuracy_lsa_3)],accuracy_lsa_3.max())
print(n_inputs[np.argmax(accuracy_dsa_3)],accuracy_dsa_3.max())
print(n_inputs[np.argmax(accuracy_random_3)],accuracy_random_3.max())
print(n_inputs[np.argmax(accuracy_nc_3)],accuracy_nc_3.max())

plt.plot(n_inputs[np.argmax(accuracy_lsa_3)],accuracy_lsa_3.max(),'-kD')
plt.plot(n_inputs[np.argmax(accuracy_dsa_3)],accuracy_dsa_3.max(),'-ko')
plt.plot(n_inputs[np.argmax(accuracy_random_3)],accuracy_random_3.max(),'-kv')
plt.plot(n_inputs[np.argmax(accuracy_nc_3)],accuracy_nc_3.max(),'-kp')

plt.xlabel('number of inputs')
plt.ylabel('accuracy')
plt.xlim([0, 37000])
#plt.ylim([0, 1])

plt.savefig("gtsrb_c1_"+"accuracy" + "_both.svg")

plt.show()

## Saving models

In [ ]:
new_model_lsa_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_lsa_e2"
i=0

for model in models_lsa:
    model.save(new_model_lsa_dir+"_"+str(i))
    i+=1

In [ ]:
new_model_dsa_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_dsa_e2"

i=0
for model in models_dsa:
    model.save(new_model_dsa_dir+"_"+str(i))
    i+=1

In [ ]:
new_model_random_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_random_e2"

i=0
for model in models_random:
    model.save(new_model_random_dir+"_"+str(i))
    i+=1

In [ ]:
new_model_nc_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_nc_e2"

i=0
for model in models_nc:
    model.save(new_model_nc_dir+"_"+str(i))
    i+=1